# **Authors:** Seth M. Woodbury (woodbuse@uw.edu) & Donghyo Kim (donghyo@uw.edu)

# **Organization & Notebook Setup/Initialization (Run Everytime)** 

In [4]:
######################################################################
### IMPORT PACKAGES & SETUP NOTEBOOK (RUN AT BEGINNING EVERY TIME) ###
######################################################################

project_name     = 'zn_hydrolase'

### MANDATORY USER SPECIFICATIONS ###
github_repo_dir  = '/home/woodbuse/publication/metallohydrolase/github/Computational_Design_of_Metallohydrolases_PrivateGitHub' # Cloned GitHub repo directory path

### SUBFOLDERS/VARIABLES (OPTIONAL: ADD EXTRA TO LIST OF DEFAULTS) ###
# 1.) subfolders to build in notebook directory
wrk_dirs_list = [
    'cmds', 'logs', 'slurm_submit', 'important_dfs', 'rfd2_configs', 'fixed_residue_jsonls',  # optionally add more input subdirectories
]

# 2.) subfolders to build in output directory
out_dirs_list = [
    'rfdiffusion2_out', 'predesign_out', 'inpainting_out', 'mpnn_out', 'af2_out', 'fastMPNNdesign',  
    'placer_input_structures','placer_out', # optionally add more output subdirectories
]

###################################################### AUTO SETUP ######################################################
### STANDARD LIBRARY IMPORTS (MAKE SURE TO USE THE SE3nv KERNEL) ###
import shlex, glob, json, math, os, random, copy, re, shutil, statistics, string, subprocess, sys
import textwrap, warnings, concurrent.futures, time, multiprocessing, itertools, operator
from pathlib import Path
from datetime import datetime
from collections import Counter

### DERIVED PATHS (CHANGE AT USER DISCRETION) ###
scripts_dir   = f'{github_repo_dir}/Scripts/'
software_dir  = f'{github_repo_dir}/Software/'
notebook_dir  = f'{github_repo_dir}/Metallohydrolase_Design_Pipeline_Tutorial/'
working_dir   = f'{notebook_dir}working_dir/'
output_dir    = f'{notebook_dir}outputs/'
theozymes_dir = f'{notebook_dir}inputs/theozymes/' # theozymes from quantum chemistry
invrot_dir    = f'{notebook_dir}inputs/invrot/'    # inverse rotamers turning quantum chemistry theozymes into PDBs while sampling different histidine ring flips and coordinations

for p in [working_dir, output_dir, scripts_dir, software_dir]: 
    Path(p).mkdir(parents=True, exist_ok=True) # ensure base dirs exist so downstream cells never fail on missing paths

### 3RD PARTY IMPORTS (MAKE SURE TO USE THE SE3nv KERNEL) ###
import numpy as np
import pandas as pd
from Bio import pairwise2
from Bio.PDB import PDBParser, PPBuilder

### CUSTOM IMPORTS ###
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)
import General_NoteBook_Functions as functions

### SETUP SUBDIRECTORIES & CREATE VARIABLES ###
functions.setup_directories(working_dir, wrk_dirs_list, export_globals=True, globals_dict=globals())
functions.setup_directories(output_dir,  out_dirs_list,  export_globals=True, globals_dict=globals())

### OPTIONALS ###
functions.set_pandas_display(all_on=True) # pandas display comfort defaults (toggle here if desired)
os.chdir(working_dir) # move into working dir for relative IO

### PRINTS ###
print(f"### PROJECT {project_name} NOTEBOOK SUCCESSFULLY INITIALIZED ON {datetime.now().strftime('%Y-%m-%d AT TIME %H:%M:%S')} ### ")
print(f"\nExample Variables & Paths:")
print(f"   working_dir   = {working_dir}")
print(f"   theozymes_dir = {theozymes_dir}")
print(f"   output_dir    = {output_dir}", f"\n")

### PROJECT zn_hydrolase NOTEBOOK SUCCESSFULLY INITIALIZED ON 2025-12-03 AT TIME 13:14:30 ### 

Example Variables & Paths:
   working_dir   = /home/woodbuse/publication/metallohydrolase/github/Computational_Design_of_Metallohydrolases_PrivateGitHub/Metallohydrolase_Design_Pipeline_Tutorial/working_dir/
   theozymes_dir = /home/woodbuse/publication/metallohydrolase/github/Computational_Design_of_Metallohydrolases_PrivateGitHub/Metallohydrolase_Design_Pipeline_Tutorial/inputs/theozymes/
   output_dir    = /home/woodbuse/publication/metallohydrolase/github/Computational_Design_of_Metallohydrolases_PrivateGitHub/Metallohydrolase_Design_Pipeline_Tutorial/outputs/ 



# **I. Generate Backbone Structures using RFD2**

## Setup & Filter RFD2

In [3]:
###################
### SET UP RFD2 ###
###################

### SET OUTPUT DIRECTORY FOR GENERATED .PDB STRUCTURES & INPUT MOTIFS + LIG NAME ###
further_prefix = '' #want to add extra prefix before '.pdb' ? If not, write ''

### SET UP THEOZYME FOR RFD2 ###
input_motif_with_residues = [f'{invrot_dir}phenylacetate_4mu_HHH_ZnO_binding_S_enantio/PSZ_H_H_H_1_63.pdb']
lig_name = "PSZ"

### SET UP CONTIGS FOR RFD2 ###
contigs = ['130-275,A1-1,B2-2,C3-3'] # ['min_length-max_length,chainXresidueY-chainXresidueY+n,...']
contig_atoms = '{"A1":"NE2,CD2,CG,CB,ND1,CE1","B2":"NE2,CD2,CG,CB,ND1,CE1","C3":"NE2,CD2,CG,CB,ND1,CE1"}' #fixed tip atoms
length_range = "130-275" # min-max length of protein, should match first range of contigs

### ESSENTIAL RFD2 PARAMETERS ###
timesteps = 75 # basically the smaller this number is, the bigger the denoising steps taken
final_step = 1 # DECIDE IF 25-50, higher = less compute time but cuts number of timesteps

nstruct = 10 # total outputs you want per topology

### BONUS FEATURES TO TOGGLE ###
guidepost_bonds = True
align_motif = False
str_self_cond = True
contig_as_guidepost = True
remove_guideposts_from_output = True
infer_guidepost_positions = True
guidepost_xyz_as_design = True
flexible_ligand = False
model_state_dict = 'model_state_dict'
recenter_xt = True
recenter_xt_dropt = '3'
deterministic = False # MAKE THIS FALSE OR ALL OUTPUTS WILL BE SAME ONLY FOR BENCHMARKING

### SET POTENTIALS? THIS MIGHT BE OUTDATED BUT CAN HELP WITH LIGAND CONTACT ###
include_potentials = False  # Set to True if you want to include the potentials section
potentials_list = ['ligand_ncontacts'] 
guide_decay = 'linear' #linear or quadratic, quadratic seems preferred these days
guide_scale = 2
weight = 0.65 #Previously 0.6

### SET SCRIPT & CHECKPOINT ###
script = './../Software/RFdiffusion2/rf_diffusion/run_inference.py' # SHOULD BE UPDATED
checkpoint = './../Software/RFdiffusion2/rf_diffusion/model_weights/RFD_140.pt'

### GENERATE OUR CONFIGS & WRITE OUR COMMANDS ###
def generate_config_content(base_name, iteration, include_potentials):
    # Start with the common configuration content
    config_content = textwrap.dedent(f"""
    defaults:
      - base

    diffuser:
      T: {timesteps}
      
    guidepost_bonds: {guidepost_bonds}

    inference:
      align_motif: {align_motif}
      ckpt_path: {checkpoint}
      num_designs: {nstruct}
      output_prefix: {rfdiffusion2_out_dir}{project_name}_{base_name}_{iteration}{further_prefix}
      input_pdb: {os.path.join(os.path.dirname(input_motif_with_residues[0]), f"{base_name}.pdb")}
      final_step: {final_step}
      ligand: {lig_name}
      contig_as_guidepost: {contig_as_guidepost}
      remove_guideposts_from_output: {remove_guideposts_from_output}
      infer_guidepost_positions: {infer_guidepost_positions}
      guidepost_xyz_as_design: {guidepost_xyz_as_design}
      flexible_ligand: {flexible_ligand}
      state_dict_to_load: {model_state_dict}
      recenter_xt: {recenter_xt}
      deterministic: {deterministic}
      recenter_xt_dropt: {recenter_xt_dropt}
      idealize_sidechain_outputs: True

    contigmap:
      contigs: {contigs}
      length: '{length_range}'
      contig_atoms: '{contig_atoms}'
    """)

    # Add the potentials section only if potentials are defined
    if include_potentials == True:
        potentials_section = textwrap.dedent(f"""
        potentials:
          guiding_potentials: ["type:{potentials_list[0]},weight:{weight}"]
          guide_scale: {guide_scale}
          guide_decay: {guide_decay}
        """)
        config_content += potentials_section
    return config_content

# Make sure to use the correct variable (include_potentials) when calling generate_config_content
print ("Command to run the script:")
for pdb in input_motif_with_residues:
    base_name = os.path.basename(pdb).replace('.pdb', '')
    config_file = f'{rfd2_configs_dir}{base_name}.yaml'
    with open(config_file, 'w') as config_f:
        config_txt = generate_config_content(base_name, 0, include_potentials)
        config_f.write(config_txt)

    cmd = f'python {script} --config-dir={rfd2_configs_dir} --config-name={base_name}.yaml \n'
    print(cmd)

Command to run the script:
python ./../Software/RFdiffusion2/rf_diffusion/run_inference.py --config-dir=./working_dir/rfd2_configs/ --config-name=PSZ_H_H_H_1_63.yaml 



In [6]:
################################
### RUN PROCESS RFD2 OUTPUTS ###
################################

# Set the multiple ligand names
multiple_ligand_names = {"PSZ"}

### INPUT STRUCTURE DIRECTORIES ###
combined_input_ligands_DIR = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/"
params_path_DIR = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/"

### OUTPUT DIRECTORIES ###
outdir = "./output/rfdiffusion2_out/"

### INPUT VARIABLES ###
SASA_lower_limit = "0"
loop_limit = "0.3"
rog = "32.0"
longest_helix = "32"
bondlen_dev = 0.3

loop_catres = "--loop_catres" # Default = True, if enabled structs with cat residues on loopy res are filtered
fix = "--fix"

# Mostly Constant
script = f"{scripts_dir}process_RFD2_outputs.py"
nproc = 19 # Number of CPUs to use

# Iterate over each ligand name to generate and print the command
for ligand_name in multiple_ligand_names:
    if len(glob.glob(f"{rfdiffusion2_out_dir}/*{ligand_name}*.pdb")) == 0:
        continue
    ### INPUT STRUCTURES ###
    pdb_path = f"{rfdiffusion2_out_dir}*{ligand_name}*.pdb"
    ref_path = f"{combined_input_ligands_DIR}*{ligand_name}*.pdb"
    params_path = f"{params_path_DIR}{ligand_name}.params"
    
    # Compile the command for each ligand
    command = (f"python {script} "
               f"--pdb {pdb_path} "
               f"--ref {ref_path} "
               f"--outdir {outdir} "
               f"--SASA_lower_limit {SASA_lower_limit} "
               f"--loop_limit {loop_limit} "
               f"--params {params_path} "
               f"--rog {rog} "
               f"--longest_helix {longest_helix} "
               f"--bondlen_dev {bondlen_dev} "
               f"{loop_catres} "
               f"{fix} "
              )
    
    # Print the command for the current ligand
    print ("Command to run the script:")
    print(command)
    print(f"mv {outdir}RFD2_analysis.sc {outdir}RFD2_analysis_{ligand_name}.sc")
    print()

Command to run the script:
python {scripts_dir}process_RFD2_outputs.py --pdb ./output/rfdiffusion2_out/*PSZ*.pdb --ref ./input/invrot/COMBINED_PHENYLACETATE_HHH/*PSZ*.pdb --outdir ./output/rfdiffusion2_out/ --SASA_lower_limit 0 --loop_limit 0.3 --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/PSZ.params --rog 32.0 --longest_helix 32 --bondlen_dev 0.3 --loop_catres --fix 
mv ./output/rfdiffusion2_out/RFD2_analysis.sc ./output/rfdiffusion2_out/RFD2_analysis_PSZ.sc



## Fix Side Chains Using Predesign on RFD2 Backbones

In [8]:
#############################################################
### RUN PREDESIGN SCRIPT TO FIX SIDE CHAINS & MINIMIZE BB ###
#############################################################

# NOTE: YOU NEED TO RUN THIS IN A SEPARATE DIRECTORY

### INPUTS ###
commands_name = "predesign_commands_after_filtered_rfd2"
ligand_names = ["PSZ"]

specific_predesign_output_dir = f'{predesign_out_dir}filtered_RFD2'
os.makedirs(specific_predesign_output_dir, exist_ok=True)

input_pdb_structures_dir = os.path.join(rfdiffusion2_out_dir, "filtered_structures/")
params_files = {ligand: glob.glob(os.path.join(theozymes_dir, "phenylacetate_4MU_substrate/HHH_to_zinc/", # Narrow it down
                                               f"**/*{ligand}*.params"), recursive=True)[0]
                for ligand in ligand_names}

cst_files = {
    ligand: os.path.join(theozymes_dir, "cst_files", f"{ligand}_3H_NO_DEPROT_RES.cst")
    for ligand in ligand_names}

### Constants ###
script_path = f"{scripts_dir}predesign_FMD.py"
scoring_path = f"{scripts_dir}scoring/esterase_scoring.py"

### Generate Commands ###
print ("Command to run the script:")
for ligand_name in ligand_names:
    pdb_files = glob.glob(os.path.join(input_pdb_structures_dir, f"*{ligand_name}*.pdb"))
    params_file = params_files[ligand_name]
    cst_file = cst_files[ligand_name]

    for pdb_file in pdb_files:
        cmd = (
            f"python {script_path} --pdb {pdb_file} "
            f"--params {params_file} --cstfile {cst_file} --output_dir {specific_predesign_output_dir} --scoring {scoring_path}"
        )
        print (cmd)

Command to run the script:
python {scripts_dir}predesign_FMD.py --pdb ./output/rfdiffusion2_out/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_0-atomized-bb-False.pdb --params ./input/theozymes/phenylacetate_4MU_substrate/HHH_to_zinc/ZnO_binding_int/S_enantio/PSZ.params --cstfile ./input/theozymes/cst_files/PSZ_3H_NO_DEPROT_RES.cst --output_dir ./output/predesign_out/filtered_RFD2 --scoring {scripts_dir}scoring/esterase_scoring.py
python {scripts_dir}predesign_FMD.py --pdb ./output/rfdiffusion2_out/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False.pdb --params ./input/theozymes/phenylacetate_4MU_substrate/HHH_to_zinc/ZnO_binding_int/S_enantio/PSZ.params --cstfile ./input/theozymes/cst_files/PSZ_3H_NO_DEPROT_RES.cst --output_dir ./output/predesign_out/filtered_RFD2 --scoring {scripts_dir}scoring/esterase_scoring.py
python {scripts_dir}predesign_FMD.py --pdb ./output/rfdiffusion2_out/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_2-atomized-bb-False.pdb --params

In [40]:
###########################################
### COMBINE ALL PREDESIGN SCORE OUTPUTS ###
###########################################

### INPUT & OUTPUT ###
scores_dir = os.path.join(predesign_out_dir, 'filtered_RFD2/scores')
output_file = os.path.join(scores_dir, "cumulative_predesign_analysis_scores.sc")

sc_files = [os.path.join(scores_dir, f) for f in os.listdir(scores_dir) if f.endswith('.sc')]
if not sc_files:
    raise ValueError("No .sc files found in the current directory.")

data_frames = []
for file in sc_files:
    if os.path.basename(file) == "cumulative_predesign_analysis_scores.sc": continue
    df = pd.read_csv(file, sep=r'\s+')
    data_frames.append(df)
    
combined_df = pd.concat(data_frames, ignore_index=True)
combined_df.to_csv(output_file, sep=' ', index=False)

# Print the real path of the output file
real_path = os.path.realpath(output_file)
print(f"The real path of the combined file is: {real_path}")

The real path of the combined file is: /home/donghyo/git/de_novo_metalloenzyme_design/output/predesign_out/filtered_RFD2/scores/cumulative_predesign_analysis_scores.sc


## Inpainting

### Setup Inpainting

In [22]:
### COPY ALL .trb TO PREDESIGN FOLDER (OR WHEREVER) ###
print("Copy .trb files to predesign folder:")
print("cp ./output/rfdiffusion2_out/filtered_structures/*.trb ./output/predesign_out/filtered_RFD2/")
print()
print("Change the name of .trb files:")
print("for f in ./output/predesign_out/filtered_RFD2/*.trb ; do mv $f ${f/.trb/_0.trb} ; done")

Copy .trb files to predesign folder:
cp ./output/rfdiffusion2_out/filtered_structures/*.trb ./output/predesign_out/filtered_RFD2/

Change the name of .trb files:
for f in ./output/predesign_out/filtered_RFD2/*.trb ; do mv $f ${f/.trb/_0.trb} ; done


In [23]:
##################################################
### GENERATE .JSON FILES TO RUN FOR INPAINTING ###
##################################################

# Set the multiple ligand names
multiple_ligand_names = {"PSZ"}

### INPUT VARIABLES ###
group = "10"  # Define your group value here
var_flag = "--var"  # Enabling variable contig lengths
ref_catres = "A1 B2 C3"  # Example format for catalytic residues
nstruct = 3 # Minimum num of structures per one

# Paths (adjust these to your actual paths)
rfd2_bbs_to_analyze_DIR = f"{predesign_out_dir}filtered_RFD2/" # MAKE SURE FILTERED
trb_dir = os.path.join(predesign_out_dir, "filtered_RFD2/") # IF not in pdb dir
params_path_DIR = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/"

# Constant Variables
script = f"{scripts_dir}setup_inpaint_from_rfd2.py"

# Iterate over each ligand name to generate and print the command
print ("Command to run the script:")
for ligand_name in multiple_ligand_names:
    ### INPUT STRUCTURES ###
    if len(glob.glob(os.path.join(rfd2_bbs_to_analyze_DIR, f"*{ligand_name}*.pdb"))) == 0:
        continue
    pdb_path = os.path.join(rfd2_bbs_to_analyze_DIR, f"*{ligand_name}*.pdb")
    trb_paths = os.path.join(trb_dir, f"*{ligand_name}*.trb")
    params_path = os.path.join(params_path_DIR, f"{ligand_name}.params")
    
    # Compile the command for each ligand
    command = (f"python {script} "
               f"--pdb {pdb_path} "
               f"--params {params_path} "
               f"--group {group} "
               f"{var_flag} "
               f"--ref_catres {ref_catres} "
               f"--trb {trb_paths} "
               f"--nstruct {nstruct} "
               f"--outdir {inpainting_out_dir}"
              )

    # Print the command for the current ligand
    print(command)
    print()
    print("Command to move the .json file:")
    print(f"for f in *.json ; do mv $f ./working_dir/cmds/${{f/cmds_/cmds{ligand_name}_}} ; done")
    print()


Command to run the script:
python {scripts_dir}setup_inpaint_from_rfd2.py --pdb ./output/predesign_out/filtered_RFD2/*PSZ*.pdb --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/PSZ.params --group 10 --var --ref_catres A1 B2 C3 --trb ./output/predesign_out/filtered_RFD2/*PSZ*.trb --nstruct 3 --outdir ./output/inpainting_out/

Command to move the .json file:
for f in *.json ; do mv $f ./working_dir/cmds/${f/cmds_/cmdsPSZ_} ; done



In [ ]:
###########################################
### MAKE INPAINTING COMMANDS AND SUBMIT ###
###########################################

# Constants
inpaint_script = f"{software_dir}proteininpainting/inpaint.py"
## Check this works. I changed the path of checkpoint. If so, remove line 66.

# Collect all JSON files in the inpainting output directory
json_files = [f for f in glob.glob("./working_dir/cmds/*.json")]

# Generate commands for each JSON file
print ("Command to run the script:")
for json_file in json_files:
    command = f"python {inpaint_script} --input_json {json_file}"
    print (command)

Command to run the script:
python {software_dir}proteininpainting/inpaint.py --input_json ./working_dir/cmds/cmdsPSZ_0.json


### Align Input Outputs with RFD2 Outputs

In [42]:
##############################################
### GENERATE COMMANDS FOR ALIGNING LIGANDS ###
##############################################

# Set the multiple ligand names
multiple_ligand_names = {"PSZ"}

### INPUT STRUCTURE DIRECTORIES ###
filtered_rfd2_bbs_as_ref_DIR = f"{predesign_out_dir}filtered_RFD2/"

combined_input_ligands_DIR = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/"
params_path_DIR = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/" #MAKE SURE params .pdb here too

aligning_inpainted_str_output_dir = f"{inpainting_out_dir}alignment/"
os.makedirs(aligning_inpainted_str_output_dir, exist_ok=True)

# Script for alignment
align_script = f"{scripts_dir}enzyme_design/align_dif_inpaint_add_ligand.py"

# Iterate over each ligand name to generate and print the command
print ("Command to run the script:")
for ligand_name in multiple_ligand_names:
    if len(glob.glob(os.path.join(inpainting_out_dir, f"*{ligand_name}*.pdb"))) == 0:
        continue
        
    ### INPUT STRUCTURES ###
    pdb_path = os.path.join(inpainting_out_dir, f"*{ligand_name}*.pdb")
    ref_path = os.path.join(rfdiffusion2_out_dir, "filtered_structures", f"*{ligand_name}*.pdb") #FILTERED DIFFUSION BBs NOW
    params_path = os.path.join(params_path_DIR, f"{ligand_name}.params")
    
    # Compile and print the command for alignment
    command_align = (f"python {align_script} "
                     f"--pdb {pdb_path} "
                     f"--ref {ref_path} "
                     f"--outdir {aligning_inpainted_str_output_dir} "
                     f"--params {params_path}")
    print(command_align)
    print()

Command to run the script:
python {scripts_dir}enzyme_design/align_dif_inpaint_add_ligand.py --pdb ./output/inpainting_out/*PSZ*.pdb --ref ./output/rfdiffusion2_out/filtered_structures/*PSZ*.pdb --outdir ./output/inpainting_out/alignment/ --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/PSZ.params



## Fix Side Chains Using Predesign on Inpainting Backbones

In [28]:
# Copy all trb files into the alignment folder
print("Command to copy all the .trb files:")
print(f"cp {inpainting_out_dir}*.trb {inpainting_out_dir}alignment")

Command to copy all the .trb files:
cp ./output/inpainting_out/*.trb ./output/inpainting_out/alignment


In [43]:
#############################################################
### RUN PREDESIGN SCRIPT TO FIX SIDE CHAINS & MINIMIZE BB ###
#############################################################

# NOTE: YOU NEED TO RUN THIS IN A SEPARATE DIRECTORY

### INPUTS ###
commands_name = "predesign_commands_after_inpainting"
ligand_names = ["PSZ"]

specific_predesign_output_dir = f'{predesign_out_dir}inpainting_outputs'
os.makedirs(specific_predesign_output_dir, exist_ok=True)

params_files = {ligand: glob.glob(os.path.join(theozymes_dir, "phenylacetate_4MU_substrate/HHH_to_zinc/", # Narrow it down
                                               f"**/*{ligand}*.params"), recursive=True)[0]
                for ligand in ligand_names}

cst_files = {
    ligand: os.path.join(theozymes_dir, "cst_files", f"{ligand}_3H_NO_DEPROT_RES.cst")
    for ligand in ligand_names}

### Constants ###
script_path = f"{scripts_dir}predesign_FMD.py"
scoring_path = f"{scripts_dir}scoring/esterase_scoring.py"

### Generate Commands ###
print ("Command to run the script:")
for ligand_name in ligand_names:
    pdb_files = glob.glob(os.path.join(inpainting_out_dir, "alignment", f"*{ligand_name}*.pdb"))
    params_file = params_files[ligand_name]
    cst_file = cst_files[ligand_name]

    for pdb_file in pdb_files:
        cmd = (
            f"python {script_path} --pdb {pdb_file} "
            f"--params {params_file} --cstfile {cst_file} --output_dir {specific_predesign_output_dir} --scoring {scoring_path}"
        )
        print (cmd)

Command to run the script:
python {scripts_dir}predesign_FMD.py --pdb ./output/inpainting_out/alignment/zn_hydrolase_PSZ_H_H_H_1_63_0_0-atomized-bb-False_0_inp_0.pdb --params ./input/theozymes/phenylacetate_4MU_substrate/HHH_to_zinc/ZnO_binding_int/S_enantio/PSZ.params --cstfile ./input/theozymes/cst_files/PSZ_3H_NO_DEPROT_RES.cst --output_dir ./output/predesign_out/inpainting_outputs --scoring {scripts_dir}scoring/esterase_scoring.py
python {scripts_dir}predesign_FMD.py --pdb ./output/inpainting_out/alignment/zn_hydrolase_PSZ_H_H_H_1_63_0_0-atomized-bb-False_0_inp_1.pdb --params ./input/theozymes/phenylacetate_4MU_substrate/HHH_to_zinc/ZnO_binding_int/S_enantio/PSZ.params --cstfile ./input/theozymes/cst_files/PSZ_3H_NO_DEPROT_RES.cst --output_dir ./output/predesign_out/inpainting_outputs --scoring {scripts_dir}scoring/esterase_scoring.py
python {scripts_dir}predesign_FMD.py --pdb ./output/inpainting_out/alignment/zn_hydrolase_PSZ_H_H_H_1_63_0_0-atomized-bb-False_0_inp_2.pdb --params .

In [13]:
###########################################
### COMBINE ALL PREDESIGN SCORE OUTPUTS ###
###########################################

### INPUT & OUTPUT ###
scores_dir = os.path.join(predesign_out_dir, 'inpainting_outputs/scores')
output_file = os.path.join(scores_dir, "cumulative_predesign_analysis_scores.sc")

sc_files = [os.path.join(scores_dir, f) for f in os.listdir(scores_dir) if f.endswith('.sc')]
if not sc_files:
    raise ValueError("No .sc files found in the current directory.")

data_frames = []
for file in sc_files:
    if os.path.basename(file) == "cumulative_predesign_analysis_scores.sc": continue
    df = pd.read_csv(file, sep=r'\s+')
    data_frames.append(df)
    
combined_df = pd.concat(data_frames, ignore_index=True)
combined_df.to_csv(output_file, sep=' ', index=False)

# Print the real path of the output file
real_path = os.path.realpath(output_file)
print(f"The real path of the combined file is: {real_path}")

The real path of the combined file is: /home/donghyo/software/Computational_Design_of_Metallohydrolases_PrivateGitHub/Metallohydrolase_Design_Pipeline_Tutorial/output/predesign_out/inpainting_outputs/scores/cumulative_predesign_analysis_scores.sc


## Filter on All Predesigned Stuctures

In [14]:
##################################################################
### FILTER THE DATA FRAME BASED ON WHATEVER CRITERIA ARE IDEAL ###
##################################################################

### HARD FILTERS ###
score_per_res = 0
all_cst = 7.25 # 7.25

### BOUNDARY FILTERS ###
L_SASA_upper = 0.5  # relative for ligand only (4MU)
L_SASA_lower = 0  # relative for ligand only (4MU)
substrate_SASA_upper = 1000 
substrate_SASA_lower = 0

subdirectories = ['filtered_RFD2', 
                  'inpainting_outputs']

dataframe_names = ['RFD2_predesign_df', 
                   'inpainting_predesign_df']

### AUTOMATED STUFF ###
def find_cumulative_files(directory, filename="cumulative_predesign_analysis_scores.sc"):
    """
    Recursively search for files with a specified name within a directory 
    and its subdirectories using glob.
    """
    file_pattern = os.path.join(directory, "**", filename)  # Recursive pattern
    return glob.glob(file_pattern, recursive=True)

dataframes = {}
for subdir, df_name in zip(subdirectories, dataframe_names):
    full_path = os.path.join(predesign_out_dir, subdir)
    found_files = find_cumulative_files(full_path)
    if found_files:
        df = pd.read_csv(found_files[0], delim_whitespace=True, comment='#')
        dataframes[df_name] = df
        globals()[df_name] = df  # Assign DataFrame to a global variable
        print(f"DataFrame {df_name} loaded with {df.shape[0]} rows and {df.shape[1]} columns.")
        print('')

            
dataframe_dict = {
    'RFD2_predesign_df': RFD2_predesign_df,
    'inpainting_predesign_df': inpainting_predesign_df
}


### FILTER AND DISPLAY RESULTS FOR EACH DATA FRAME ###
filtered_dataframes = {}

for df_name, df in dataframe_dict.items():
    filtered_df_name = 'filtered_' + df_name
    filter_mask = ((df['all_cst'] <= all_cst) & 
                   (df['score_per_res'] <= score_per_res) & 
                   (df['L_SASA'] <= L_SASA_upper) & 
                   (df['L_SASA'] >= L_SASA_lower) & 
                   (df['substrate_SASA'] <= substrate_SASA_upper) & 
                   (df['substrate_SASA'] >= substrate_SASA_lower))
    
    filtered_df = df[filter_mask].copy()
    globals()[filtered_df_name] = filtered_df
    filtered_dataframes[filtered_df_name] = filtered_df
    filter_rate = float(len(filtered_df)) / float(len(df)) * 100.0

    # Output results
    print(f'**{len(filtered_df)} of {len(df)} ({round(filter_rate, 3)}%) passed in {filtered_df_name}**')

DataFrame RFD2_predesign_df loaded with 4 rows and 44 columns.

DataFrame inpainting_predesign_df loaded with 17 rows and 44 columns.

**2 of 4 (50.0%) passed in filtered_RFD2_predesign_df**
**4 of 17 (23.529%) passed in filtered_inpainting_predesign_df**


In [70]:
###########################################
### COPY FILTERED PREDESIGNED BACKBONES ###
###########################################

for i, (df_name, filtered_df) in enumerate(filtered_dataframes.items()):
    print (df_name)
    filtered_predesign_path = os.path.join(predesign_out_dir, subdirectories[i], "filtered_structures")
    if not os.path.isdir(filtered_predesign_path):
        os.mkdir(filtered_predesign_path)
    
    print (f"Number of PDB files to copy: {len(filtered_df)}")
    for i, desc in enumerate(filtered_df["description"]):
        shutil.copy(desc+".pdb", os.path.join(filtered_predesign_path, os.path.basename(desc)+".pdb"))
        print(os.path.join(filtered_predesign_path, os.path.basename(desc)+".pdb"))
        if i % 1000 == 0:
            print (f"[{time.ctime()}] {i}th PDB file was copied...")
    print ("Done")

filtered_RFD2_predesign_df
Number of PDB files to copy: 2
./output/predesign_out/filtered_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0.pdb
[Mon Apr 21 17:26:43 2025] 0th PDB file was copied...
./output/predesign_out/filtered_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_2-atomized-bb-False_0.pdb
Done
filtered_inpainting_predesign_df
Number of PDB files to copy: 4
./output/predesign_out/inpainting_outputs/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_0-atomized-bb-False_0_inp_2_0.pdb
[Mon Apr 21 17:26:43 2025] 0th PDB file was copied...
./output/predesign_out/inpainting_outputs/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_0-atomized-bb-False_0_inp_3_0.pdb
./output/predesign_out/inpainting_outputs/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_0-atomized-bb-False_0_inp_4_0.pdb
./output/predesign_out/inpainting_outputs/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_inp_3_0.pdb
Done


# **II. ProteinMPNN to Generate Sequences for AF2**

## Run ProteinMPNN

In [11]:
##################################################################
### GRAB CATALYTIC RESIDUES AND MAKE FIXED POS .JSONL FOR MPNN ###
##################################################################

### INPUTS FOR THE FUNCTION ###
input_pdb_directory =  f"{predesign_out_dir}filtered_RFD2/filtered_structures/"
# num 1: f"{predesign_out_dir}filtered_RFD2/filtered_structures/"
# num 2: f"{predesign_out_dir}inpainting_outputs/filtered_structures/"
output_jsonl = f"{fixed_residue_jsonls_dir}filtered_RFD2.jsonl"
# num 1: f"{fixed_residue_jsonls_dir}filtered_RFD2.jsonl"
# num 2: f"{fixed_residue_jsonls_dir}inpainting.jsonl"

# Constants
script_path = f"{scripts_dir}grab_cat_residues_from_pdb_and_write_fixedAA_jsonl.py"

# Constructing the command
command = (f"python {script_path} --input_dir {input_pdb_directory} "
           f"--output_jsonl {output_jsonl}"
          )
# Print the command
print("Command to run the script:")
print(command)

Command to run the script:
python {scripts_dir}grab_cat_residues_from_pdb_and_write_fixedAA_jsonl.py --input_dir ./output/predesign_out/filtered_RFD2/filtered_structures/ --output_jsonl ./working_dir/fixed_residue_jsonls/filtered_RFD2.jsonl


In [16]:
####################################################################
### RUN MPNN FOR GENERATION X OF STRUCTURES | X = NATURAL NUMBER ###
####################################################################

filtered_RFD2_predesign_df['pdb_path'] = filtered_RFD2_predesign_df['description'] + '.pdb'
filtered_inpainting_predesign_df['pdb_path'] = filtered_inpainting_predesign_df['description'] + '.pdb'

### INPUT STRUCTURES FOR FILTERED DF ###
input_mpnn_pdb_FILTERED_DF = filtered_RFD2_predesign_df
# num 1: "filtered_RFD2_predesign_df
# num 2: "filtered_inpainting_predesign_df

pdb_path_column_name = 'pdb_path'

### INPUT .jsonl ###
fixed_res_jsonl_basename = "filtered_RFD2.jsonl" #CHANGE EVERY TIME
# num 1: "filtered_RFD2.jsonl
# num 2: "inpainting.jsonl
formatted_fixed_AA_jsonl = os.path.join(fixed_residue_jsonls_dir, fixed_res_jsonl_basename)

# options for MPNN sequence design
nstruct = 10              # num1: 10, num2: 5, num3: 5
fused_mpnn_batch_size = 1
temp = 0.1                # num1: 0.1, num2: 0.3, num3: 0.6
omit_aa = 'MCX'
pack = 0

# Where to write commands & output mpnn (mostly unchanged now)
updated_mpnn_out_dir = os.path.join(mpnn_out_dir, "predesigned_RFD2")
# num 1: "predesigned_RFD2
# num 2: "predesigned_inpainting
os.makedirs(updated_mpnn_out_dir, exist_ok=True)

# Parameters for fused_mpnn
packed_suffix = f'_packed_tempX10_{int(temp*10)}_'
mpnn_design_script = './../Software/fastmpnndesign/lib/LigandMPNN/run.py'
model_params_path = "./../Software/fastmpnndesign/lib/LigandMPNN/model_params/"
specific_model_type_for_fused_mpnn = 'protein_mpnn'
bias_AA = "A:-0.75"
sc_num_denoising_steps = 6 
repack_everything = 0
save_stats = 0 

# generate commands

for pdb in input_mpnn_pdb_FILTERED_DF[f'{pdb_path_column_name}']: #FOR DATA FRAME
    pdb = os.path.join(os.path.dirname(pdb), "filtered_structures", os.path.basename(pdb))
    cmd = f'python {mpnn_design_script} \
    --model_type {specific_model_type_for_fused_mpnn} \
    --omit_AA {omit_aa} \
    --pdb_path {pdb} \
    --out_folder {updated_mpnn_out_dir} \
    --number_of_batches {nstruct} \
    --temperature {temp} \
    --batch_size {fused_mpnn_batch_size} \
    --pack_side_chains {pack} \
    --fixed_residues_multi {formatted_fixed_AA_jsonl} \
    --sc_num_denoising_steps {sc_num_denoising_steps} \
    --repack_everything {repack_everything} \
    --packed_suffix {packed_suffix} \
    --bias_AA {bias_AA} \
    --save_stats {save_stats} \
    --checkpoint_protein_mpnn {os.path.join(model_params_path, "proteinmpnn_v_48_020.pt")} \
    --checkpoint_ligand_mpnn {os.path.join(model_params_path, "ligandmpnn_v_32_010_25.pt")} \
    --checkpoint_path_sc {os.path.join(model_params_path, "ligandmpnn_sc_v_32_002_16.pt")}'
    print(cmd)
        

python ./../Software/fastmpnndesign/lib/LigandMPNN/run.py     --model_type protein_mpnn     --omit_AA MCX     --pdb_path ./output/predesign_out/filtered_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0.pdb     --out_folder ./output/mpnn_out/predesigned_RFD2     --number_of_batches 10     --temperature 0.1     --batch_size 1     --pack_side_chains 0     --fixed_residues_multi ./working_dir/fixed_residue_jsonls/filtered_RFD2.jsonl     --sc_num_denoising_steps 6     --repack_everything 0     --packed_suffix _packed_tempX10_1_     --bias_AA A:-0.75     --save_stats 0     --checkpoint_protein_mpnn ./../Software/fastmpnndesign/lib/LigandMPNN/model_params/proteinmpnn_v_48_020.pt     --checkpoint_ligand_mpnn ./../Software/fastmpnndesign/lib/LigandMPNN/model_params/ligandmpnn_v_32_010_25.pt     --checkpoint_path_sc ./../Software/fastmpnndesign/lib/LigandMPNN/model_params/ligandmpnn_sc_v_32_002_16.pt
python ./../Software/fastmpnndesign/lib/LigandMPNN/run.py     --mode

## Paste Remark 666 Lines into MPNN & Make Updated .jsonl with Catalytic Residues

In [84]:
#########################################################################################
### TRANSFER REMARK 666 + IMPORTANT LINES FROM PDBS IN ONE DIRECTORY TO NEW DIRECTORY ###
#########################################################################################

### INPUT DIRECTORIES ###
new_pdb_dir = f'{mpnn_out_dir}predesigned_RFD2/backbones/' #pdbs further in pipeline
#num 1: f'{mpnn_out_dir}predesigned_RFD2/backbones/'
#num 2: f'{mpnn_out_dir}predesigned_inpainting/backbones/'

reference_pdb_dir = f'{predesign_out_dir}filtered_RFD2/' #og pdbs
#num 1: f'{predesign_out_dir}filtered_RFD2/'
#num 2: f'{predesign_out_dir}inpainting_outputs/'

sample_size = 60

### CONSTANTS ###
script = '{scripts_dir}add_remark666_lines_to_pdb.py'

### AUTOMATED COMMAND GENERATION + PRINT ###
command = (f"python {script} "
           f"--new_pdb_dir {new_pdb_dir} "
           f"--reference_old_pdb_dir {reference_pdb_dir} "
           f"--find_suffixes_from_random_sample {sample_size} "
           f"--debug " #verbose
          )

print("Command to execute:")
print(command)

Command to execute:
python {scripts_dir}add_remark666_lines_to_pdb.py --new_pdb_dir ./output/mpnn_out/predesigned_RFD2/backbones/ --reference_old_pdb_dir ./output/predesign_out/filtered_RFD2/ --find_suffixes_from_random_sample 60 --debug 


# **III. AlphaFold2 (1st Iteration)**

## Run AF2

To run SuperFold, you need to download AlphaFold2 model parameters.

📍 Steps:
1. Make sure `aria2c` is installed:
   ```bash
   sudo apt install aria2
   ```
2. Download the parameters by running:
   ```bash
   bash {software_dir}superfold/scripts/download_all_data.sh {software_dir}superfold/alphafold_weights.pth
   ```

NOTE: AlphaFold2 parameters are provided under a non-commercial CC BY-NC 4.0 license.

In [ ]:
#######################################
### predict designs with AlphaFold2 ###
#######################################

### CONSTANTS ###
#apptainer = './containers/crispy.sif' 
apptainer = 'python' 
#superfold = '{software_dir}superfold/run_superfold.py'
superfold = '{software_dir}superfold/run_superfold.py'

### CHANGE THESE EVERY TIME ###
af2_inputs_directory = f'{mpnn_out_dir}predesigned_RFD2/backbones/'
#num 1: f'{mpnn_out_dir}predesigned_RFD2/backbones/'
#num 2: f'{mpnn_out_dir}predesigned_inpainting/backbones/'

af2_output_dir = f'{af2_out_dir}predesigned_RFD2/'
#num 1: f'{af2_out_dir}predesigned_RFD2/'
#num 2: f'{af2_out_dir}predesigned_inpainting/'
os.makedirs(af2_output_dir, exist_ok=True)

### ALPHAFOLD PARAMETERS ###
model_number = 4 # 4 is recommended, 5 is probably similar
model_type = 'monomer_ptm'
version = 'monomer'

nstruct = 1 # how many structures to generate as outputs, default = 1
num_recycles = 6 # default = 3
num_ensemble = 2 # default in code is 1 but CASP competition used 8
mock_msa_depth = 1 # default is 1, but higher might = better ? goes to +inf
recycle_tol = 0.0 # default is 0.0, what convergence rmsd to stop at after recycling

pct_seq_mask = 0.15 # 0.15 is default, percent of sequence to make during inference
output_summary = '' #'--output_summary' # ENABLE IF YES, otherwise ''
output_pae = '--output_pae' # ENABLE IF YES, otherwise ''
amber_relax = '' #'--amber_relax' # ENABLE IF YES, otherwise ''

### MAKE THE FUNCTION ###
cmd = f'{apptainer} {superfold} --out_dir {af2_output_dir} --model {model_number} --type {model_type} --version {version} \
--nstruct {nstruct} --max_recycles {num_recycles} --num_ensemble {num_ensemble} --mock_msa_depth {mock_msa_depth} --recycle_tol {recycle_tol} \
--pct_seq_mask {pct_seq_mask} {output_summary} {output_pae} {amber_relax}' #{initial_guess}'

for i, pdb in enumerate(glob.glob(os.path.join(af2_inputs_directory, f"*.pdb"))): 
    if i == 0 or i % 5!= 0:
        cmd += f" {pdb}"
    else:
        print(cmd)
        cmd = f'{apptainer} {superfold} --out_dir {af2_output_dir} --model {model_number} \
        --type {model_type} --version {version} \
        --nstruct {nstruct} --max_recycles {num_recycles} --num_ensemble {num_ensemble} \
        --mock_msa_depth {mock_msa_depth} --recycle_tol {recycle_tol} \
        --pct_seq_mask {pct_seq_mask} {output_summary} {output_pae} {amber_relax} \
        {pdb}'
print(cmd)

python {software_dir}superfold/run_superfold.py --out_dir ./output/af2_out/predesigned_RFD2/ --model 4 --type monomer_ptm --version monomer --nstruct 1 --max_recycles 6 --num_ensemble 2 --mock_msa_depth 1 --recycle_tol 0.0 --pct_seq_mask 0.15  --output_pae  ./output/mpnn_out/predesigned_RFD2/backbones/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_1.pdb ./output/mpnn_out/predesigned_RFD2/backbones/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10.pdb ./output/mpnn_out/predesigned_RFD2/backbones/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_2.pdb ./output/mpnn_out/predesigned_RFD2/backbones/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_3.pdb ./output/mpnn_out/predesigned_RFD2/backbones/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_4.pdb
python {software_dir}superfold/run_superfold.py --out_dir ./output/af2_out/predesigned_RFD2/ --model 4         --type monomer_ptm --version monomer         --nstruct 1 --max_recycles 6 --num_ensemble 2         --mock_msa_depth

## Analyze AF2

In [4]:
##################################################################
### PARSE & PROCESS AF2 OUTPUT FILES ONLY NEED TO DO THIS ONCE ###
##################################################################

# INPUT DIRECTORY TO PARSE
af2_dir = f'{af2_out_dir}predesigned_RFD2/'
#num 1: f'{af2_out_dir}predesigned_RFD2/'
#num 2: f'{af2_out_dir}predesigned_inpainting/'

# Constants
script = f"{scripts_dir}af2_parse_processing_multi.py"
optional_cpus = 19  # Optional: specify the number of CPUs to use, or remove this variable to use default logic

# Make command
command = f"python {script} {af2_dir}"
if 'optional_cpus' in locals():
    command += f" --cpus {optional_cpus}"

# Print submission
print("SUBMIT THIS:")
print(command)
print('')

# Print output .csv path
print("OUTPUT CSV FILE HERE:")
print(af2_dir + 'af2_out_parsed.csv')

SUBMIT THIS:
python {scripts_dir}af2_parse_processing_multi.py ./output/af2_out/predesigned_RFD2/ --cpus 19

OUTPUT CSV FILE HERE:
./output/af2_out/predesigned_RFD2/af2_out_parsed.csv


## Filter AF2 structures based on Global Metric

In [7]:
############################################
### LOAD AND FILTER AF2 BY SUMMARY STATS ###
############################################

# Correct the path to where your actual CSV file is located
csv_file_path = f'{af2_out_dir}predesigned_RFD2/af2_out_parsed.csv'
#num 1: f'{af2_out_dir}predesigned_RFD2/'
#num 2: f'{af2_out_dir}predesigned_inpainting/'

output_filtered_structures_dir = f'{af2_out_dir}predesigned_RFD2/filtered_structures/'
#num 1: f'{af2_out_dir}predesigned_RFD2/filtered_structures/'
#num 2: f'{af2_out_dir}predesigned_inpainting/filtered_structures/'

# Select parameters for filtering -> EXTREMELY HARSH IN THIS CASE
plddt_cutoff = 60 # in pLDDT units -> usually 70-80
rmsd_cutoff = 3 # Å, I think this is Cα (1.5 is typical, 2.0 might be a stretch but okay for big structures)
pae_cutoff = 100 # below 5 is harsh but good
pTM_cutoff = 0.5 # Supposedly above 0.75 is good
convergence_cutoff = 1.5

# Load the CSV file into a DataFrame
af2_scores_df = pd.read_csv(csv_file_path)

af2_mask = ((af2_scores_df['mean_plddt'] >= plddt_cutoff) & 
            (af2_scores_df['rmsd_to_input'] <= rmsd_cutoff) &
            (af2_scores_df['mean_pae'] <= pae_cutoff) & 
            (af2_scores_df['pTMscore'] >= pTM_cutoff) &
            (af2_scores_df['af2_convergence_tol'] <= convergence_cutoff)
           )

filtered_af2_scores_df = af2_scores_df[af2_mask]
af2_filter_rate = float(len(filtered_af2_scores_df))/float(len(af2_scores_df))*100.0
print(f'{len(filtered_af2_scores_df)} of {len(af2_scores_df)} ({round(af2_filter_rate,3)}%) with plddt >= {plddt_cutoff} and Cα rmsd <={rmsd_cutoff}')

### FUNCTION TO RUN ###
def copy_files(df, pdb_col, json_col, output_dir):
    existing_files = os.listdir(output_dir)
    print(f"Initial file count in the directory (Should = 0): {len(existing_files)}")

    count = 0
    start_time = time.time()
    error_files = []

    for index, row in df.iterrows():
        pdb_path = row[pdb_col]
        json_path = row[json_col]

        try:
            shutil.copy(pdb_path, output_dir)
            shutil.copy(json_path, output_dir)
            count += 2  # Increment for each PDB and JSON file copied

            if count % 2000 == 0:
                current_time = time.ctime()
                elapsed_time = time.time() - start_time
                files_left = (len(df)*2 - count)
                average_time_per_file = elapsed_time / count
                estimated_time_remaining = max(0, average_time_per_file * files_left)
                print(f"[{current_time}] {count} files copied successfully. Elapsed time: {elapsed_time:.2f} seconds. Average time per file: {average_time_per_file:.4f} seconds. Estimated time remaining: {estimated_time_remaining:.2f} seconds.")

        except Exception as e:
            error_files.append((pdb_path, json_path, str(e)))

    print(f"Total files copied: {count}")
    if error_files:
        print("Failed to copy the following files:")
        for error in error_files:
            print(error)

### RUN THE SCRIPT ###
os.makedirs(output_filtered_structures_dir, exist_ok=True)
copy_files(filtered_af2_scores_df, 'pdb_path', 'af2_json_path', output_filtered_structures_dir)
print('')
print('Done!')

10 of 20 (50.0%) with plddt >= 60 and Cα rmsd <=3
Initial file count in the directory (Should = 0): 20
Total files copied: 20

Done!


## Analyze AF2 on Side Chain Metrics & Do Global Ligand Alignment

### Run Both of These Once to Generate Score Files

In [10]:
##################################################################
### PARSE & PROCESS AF2 OUTPUT FILES ONLY NEED TO DO THIS ONCE ###
##################################################################

# INPUT DIRECTORY TO PARSE
af2_directory_to_parse = f'{af2_out_dir}predesigned_RFD2/filtered_structures/'
#num 1: f'{af2_out_dir}predesigned_RFD2/filtered_structures/'
#num 2: f'{af2_out_dir}predesigned_inpainting/filtered_structures/'

# Constants
script = f"{scripts_dir}af2_parse_processing_multi.py"
optional_cpus = 19  # Optional: specify the number of CPUs to use, or remove this variable to use default logic
optional_basename_for_summary_stats = "scores.sc"

# Make command
command = f"python {script} {af2_directory_to_parse}"
if 'optional_cpus' in locals():
    command += f" --cpus {optional_cpus}"
if 'optional_basename_for_summary_stats' in locals():
    command += f" --optional_basename_for_summary_stats {optional_basename_for_summary_stats}"

# Print submission
print("SUBMIT THIS:")
print(command)
print('')

# Print output .csv path
print("OUTPUT CSV FILE HERE:")
print(af2_directory_to_parse + f'{optional_basename_for_summary_stats}')

SUBMIT THIS:
python {scripts_dir}af2_parse_processing_multi.py ./output/af2_out/predesigned_RFD2/filtered_structures/ --cpus 19 --optional_basename_for_summary_stats scores.sc

OUTPUT CSV FILE HERE:
./output/af2_out/predesigned_RFD2/filtered_structures/scores.sc


In [14]:
##################################################################################################
### CALCULATE SIDE CHAIN RMSDS & OTHER RMSDS FOR CATALYTIC RESIDUES IN FILTERED STRUCTURES DIR ###
##################################################################################################

### INPUT SCORE PATH + OUTPUT SCORE PATH & REFERENCE FILES ###
score_path = os.path.join(f'{af2_out_dir}predesigned_RFD2/filtered_structures', 'scores.sc')
output_file = os.path.join(f'{af2_out_dir}predesigned_RFD2/filtered_structures', 'updated_scores.sc')
#num 1: f'{af2_out_dir}predesigned_RFD2/filtered_structures'
#num 2: f'{af2_out_dir}predesigned_inpainting/filtered_structures'

ref_pdbs_dir = os.path.join(f'{mpnn_out_dir}predesigned_RFD2', 'backbones/')
#num 1: f'{mpnn_out_dir}predesigned_RFD2/backbones/'
#num 2: f'{mpnn_out_dir}predesigned_inpainting/backbones/'

### INPUT PARAMS ###
params_path = os.path.join(invrot_dir, "COMBINED_PHENYLACETATE_HHH", "COMBINED_PARAMS", "*.params")  # Ensure params include .pdb if needed

### INPUT ATOM RMSDS OF CAT RESIDUES TO CALCULATE & NUM OF SUFFIXES ###
atom_groups_json = { #MANUALLY INPUT POSSIBLE CATALYTIC RESIDUES!!
    "HIS": [
        {"atoms": ["N", "CA", "C", "O"], "label": "bb_rmsd"},
        {"atoms": ["N", "CA", "C", "O", "CB"], "label": "cb_bb_rmsd"},
        {"atoms": ["CG", "CE1", "ND1", "NE2", "CD2"], "label": "imidazole_rmsd"},
        {"atoms": ["CA", "CB"], "label": "ca_cb_rmsd"},
        {"atoms": ["ND1", "NE2"], "label": "imid_sc_nitrogen_rmsd"}
    ]
}

max_possible_num_of_suffixes = 1
column_name_for_pdb_path = 'pdb_path'

### CONSTANTS ###
script = '{scripts_dir}sidechain_rmsd_and_info_af2_matching_res.py'
atom_groups = json.dumps(atom_groups_json)

# Build the command
command = (
    f'python {script} --scorefile {score_path} --ref_pdbs_dir {ref_pdbs_dir} --params {params_path} --output_file {output_file} '
    f'--max_possible_num_of_suffixes {max_possible_num_of_suffixes} --atom_groups \'{atom_groups}\''
)
print("Command to execute:")
print(command)
print('')
print('New scorefile:')
print(f'{output_file}')

Command to execute:
python {scripts_dir}sidechain_rmsd_and_info_af2_matching_res.py --scorefile ./output/af2_out/predesigned_RFD2/filtered_structures/scores.sc --ref_pdbs_dir ./output/mpnn_out/predesigned_RFD2/backbones/ --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/*.params --output_file ./output/af2_out/predesigned_RFD2/filtered_structures/updated_scores.sc --max_possible_num_of_suffixes 1 --atom_groups '{"HIS": [{"atoms": ["N", "CA", "C", "O"], "label": "bb_rmsd"}, {"atoms": ["N", "CA", "C", "O", "CB"], "label": "cb_bb_rmsd"}, {"atoms": ["CG", "CE1", "ND1", "NE2", "CD2"], "label": "imidazole_rmsd"}, {"atoms": ["CA", "CB"], "label": "ca_cb_rmsd"}, {"atoms": ["ND1", "NE2"], "label": "imid_sc_nitrogen_rmsd"}]}'

New scorefile:
./output/af2_out/predesigned_RFD2/filtered_structures/updated_scores.sc


### Do a Light Side Chain Filter Before Global Alignment & Harsh SC Filter

In [6]:
##################################################################################
### COMPUTE SIDE CHAIN STATISTICS FROM SCORES DF & DO GENTLE/LOGICAL FILTERING ###
##################################################################################

### INPUT PATH ###
scores_filepath = f"{af2_out_dir}predesigned_RFD2/filtered_structures/updated_scores.sc"
#num 1: f'{af2_out_dir}predesigned_RFD2/filtered_structures/updated_scores.sc'
#num 2: f'{af2_out_dir}predesigned_inpainting/filtered_structures/updated_scores.sc'
inpaint_or_diffusion = "RFD2" #RFD2 or Inpainting

### INPUT SPECIFY COLUMN GROUPS FOR CALCULATION ###
column_groups = {
    'cat_HIS_bb': ['cat_HIS1_bb_rmsd', 'cat_HIS2_bb_rmsd', 'cat_HIS3_bb_rmsd'],
    'cat_HIS_cb_bb': ['cat_HIS1_cb_bb_rmsd', 'cat_HIS2_cb_bb_rmsd', 'cat_HIS3_cb_bb_rmsd'],
    'cat_HIS_ca_cb_ONLY': ['cat_HIS1_ca_cb_rmsd', 'cat_HIS2_ca_cb_rmsd', 'cat_HIS3_ca_cb_rmsd'],
    'cat_HIS_imidazole': ['cat_HIS1_imidazole_rmsd', 'cat_HIS2_imidazole_rmsd', 'cat_HIS3_imidazole_rmsd'],
    'cat_HIS_imidazole_tip_Ns': ['cat_HIS1_imid_sc_nitrogen_rmsd', 'cat_HIS2_imid_sc_nitrogen_rmsd', 'cat_HIS3_imid_sc_nitrogen_rmsd']
}

### FUNCTION TO CALCULATE MEAN RMSD AND RMS OF RMSDs FOR SPECIFIED COLUMN GROUPS ###
def calculate_group_rmsd(df, column_groups):
    for group_name, columns in column_groups.items():
        mean_col_name = f"{group_name}_mean_rmsd"
        rms_col_name = f"{group_name}_rms_of_rmsds"
        df[mean_col_name] = df[columns].mean(axis=1)
        df[rms_col_name] = np.sqrt((df[columns]**2).mean(axis=1))
    print (df)

### EXECUTE PRINTING ###
sidechain_rmsd_df = pd.read_csv(scores_filepath, sep=',', na_values='')
calculate_group_rmsd(sidechain_rmsd_df, column_groups)

                                                                                                                                             af2_json_path  \
0   ./output/af2_out/predesigned_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_7_model_4_ptm_seed_0_prediction_results.json   
1  ./output/af2_out/predesigned_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_prediction_results.json   
2   ./output/af2_out/predesigned_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_6_model_4_ptm_seed_0_prediction_results.json   
3   ./output/af2_out/predesigned_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_1_model_4_ptm_seed_0_prediction_results.json   
4   ./output/af2_out/predesigned_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_2_model_4_ptm_seed_0_prediction_results.json   
5   ./output/af2_out/predesigned_RFD2/filtered_struc

In [7]:
##################################################################################
### COMPUTE SIDE CHAIN STATISTICS FROM SCORES DF & DO GENTLE/LOGICAL FILTERING ###
##################################################################################

### NEW FILTER FROM SIDE CHAIN 2 ###
cat_HIS_bb_rms_of_rmsds_cutoff = 2 #2 # Å
cat_HIS_cb_bb_rms_of_rmsds_cutoff = 2 # Å
cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff = 2 # Å
cat_HIS_imidazole_rms_of_rmsds_cutoff = 3 # Å
cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff = 3 # Å

# Individual filter calculations with specific naming
filter_masks = {
    'cat_HIS_bb_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_bb_rms_of_rmsds'] <= cat_HIS_bb_rms_of_rmsds_cutoff,
    'cat_HIS_cb_bb_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_cb_bb_rms_of_rmsds'] <= cat_HIS_cb_bb_rms_of_rmsds_cutoff,
    'cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_ca_cb_ONLY_rms_of_rmsds'] <= cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff,
    'cat_HIS_imidazole_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_imidazole_rms_of_rmsds'] <= cat_HIS_imidazole_rms_of_rmsds_cutoff,
    'cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_imidazole_tip_Ns_rms_of_rmsds'] <= cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff
}

# Calculate pass counts and rates for each filter
filter_results = {filter_name: (mask.sum(), len(sidechain_rmsd_df), mask.mean() * 100) for filter_name, mask in filter_masks.items()}

# Overall filtering result using OR logic
cumulative_filter_mask_or = filter_masks['cat_HIS_bb_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_cb_bb_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_imidazole_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask']

cumulative_pass_count_or = cumulative_filter_mask_or.sum()
cumulative_total_or = len(sidechain_rmsd_df)
cumulative_pass_rate_or = cumulative_pass_count_or / cumulative_total_or * 100

### DO THE PRINT STATEMENT (FANCY) ###
print('########################## SIDE CHAIN RMSD METRICS FILTER ##########################')
for filter_name, (pass_count, total, pass_rate) in filter_results.items():
    specific_criteria = filter_name.replace('_mask', '').replace('_', ' ').replace('cat', 'catalytic').upper()
    cutoff_value = {
        'cat_HIS_bb_rms_of_rmsds_mask': cat_HIS_bb_rms_of_rmsds_cutoff,
        'cat_HIS_cb_bb_rms_of_rmsds_mask': cat_HIS_cb_bb_rms_of_rmsds_cutoff,
        'cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask': cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff,
        'cat_HIS_imidazole_rms_of_rmsds_mask': cat_HIS_imidazole_rms_of_rmsds_cutoff,
        'cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask': cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff
    }[filter_name]
    print(f'{pass_count} of {total} ({pass_rate:.3f}%) have {specific_criteria} <= {cutoff_value}')

# Maintaining the cumulative print statement from the provided script
print('')
print(f'{cumulative_pass_count_or} of {cumulative_total_or} ({cumulative_pass_rate_or:.3f}%) pass at least one filter!')
print('####################################################################################')

### MAKE THE FINAL DATA FRAME ###
final_filtered_df = sidechain_rmsd_df[cumulative_filter_mask_or]

### PRINT OUT COLUMN NAMES USED IN FILTERING ###
filtered_columns = ', '.join([key.replace('_mask', '') for key in filter_masks.keys()])
print(f'Columns used in filtering: {filtered_columns}')

########################## SIDE CHAIN RMSD METRICS FILTER ##########################
8 of 10 (80.000%) have CATALYTIC HIS BB RMS OF RMSDS <= 2
8 of 10 (80.000%) have CATALYTIC HIS CB BB RMS OF RMSDS <= 2
8 of 10 (80.000%) have CATALYTIC HIS CA CB ONLY RMS OF RMSDS <= 2
8 of 10 (80.000%) have CATALYTIC HIS IMIDAZOLE RMS OF RMSDS <= 3
8 of 10 (80.000%) have CATALYTIC HIS IMIDAZOLE TIP NS RMS OF RMSDS <= 3

8 of 10 (80.000%) pass at least one filter!
####################################################################################
Columns used in filtering: cat_HIS_bb_rms_of_rmsds, cat_HIS_cb_bb_rms_of_rmsds, cat_HIS_ca_cb_ONLY_rms_of_rmsds, cat_HIS_imidazole_rms_of_rmsds, cat_HIS_imidazole_tip_Ns_rms_of_rmsds


### Create Alignment with Lightly Filtered Structures

In [8]:
##############################################
### GENERATE COMMANDS FOR ALIGNING LIGANDS ###
##############################################

import time

### INPUTS ###
column_of_pdb_paths = 'pdb_path' # COLUMN OF DF W/PDB PATHS

ref_pdbs_to_match = f'{predesign_out_dir}filtered_RFD2/'
#num1: f'{predesign_out_dir}filtered_RFD2/'
#num2: f'{predesign_out_dir}inpainting_outputs/'

params_path = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/"

align_af2_out_directory = os.path.join(f"{af2_out_dir}", 
                                       "predesigned_RFD2", # num1: predesigned_RFD2, num2: predesigned_inpainting
                                       "filtered_structures",
                                       "alignment/")
### CONSTANTS ###
align_script = f"{scripts_dir}align_af2_with_inputs_and_copy_ligand.py"

### REFERENCE PDB PROCESSING ###
ref_pdb_files = glob.glob(os.path.join(ref_pdbs_to_match, '*.pdb'))
ref_pdb_map = {os.path.basename(p).replace('.pdb', ''): p for p in ref_pdb_files}
ref_pdb_usage = Counter()

### AUTOMATED COMMAND WRITING & REFERENCE MATCHING ###
os.makedirs(align_af2_out_directory, exist_ok=True)
max_usages_per_ref_pdb = 30  # Number of times a reference pdb can be used

total_rows = len(final_filtered_df)
rows_processed = 0
update_interval = 1000

start_time, num_cmds = time.time(), 0
for index, row in final_filtered_df.iterrows():
    pdb_path = row[column_of_pdb_paths]
    pdb_name = os.path.basename(pdb_path).split('.')[0]

    # Find matching reference PDB
    matching_ref = None
    for ref_name, ref_path in ref_pdb_map.items():
        if pdb_name.startswith(ref_name) and ref_pdb_usage[ref_name] < max_usages_per_ref_pdb:
            matching_ref = ref_path
            ref_pdb_usage[ref_name] += 1
            if ref_pdb_usage[ref_name] >= max_usages_per_ref_pdb:
                del ref_pdb_map[ref_name]  # Remove from future considerations
            break

    if matching_ref:
        command_align = (f"python {align_script} "
                         f"--pdb {pdb_path} "
                         f"--ref {matching_ref} "
                         f"--outdir {align_af2_out_directory} "
                         f"--params {os.path.join(params_path, '*.params')} "
                         f"--fix_catres")
        print (command_align)
        num_cmds += 1

    rows_processed += 1
    if rows_processed % update_interval == 0 or rows_processed == total_rows:
        elapsed_time = time.time() - start_time
        time_per_row = elapsed_time / rows_processed
        estimated_time_remaining = time_per_row * (total_rows - rows_processed)
        print(f"{rows_processed} rows processed, {total_rows - rows_processed} remaining. "
              f"{len(ref_pdb_map)} ref pdb files available for matching. "
              f"Elapsed time: {elapsed_time:.2f} s. Estimated time left: {estimated_time_remaining:.2f} s.")

python {scripts_dir}align_af2_with_inputs_and_copy_ligand.py --pdb ./output/af2_out/predesigned_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_7_model_4_ptm_seed_0_unrelaxed.pdb --ref ./output/predesign_out/filtered_RFD2/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0.pdb --outdir ./output/af2_out/predesigned_RFD2/filtered_structures/alignment/ --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/*.params --fix_catres
python {scripts_dir}align_af2_with_inputs_and_copy_ligand.py --pdb ./output/af2_out/predesigned_RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed.pdb --ref ./output/predesign_out/filtered_RFD2/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0.pdb --outdir ./output/af2_out/predesigned_RFD2/filtered_structures/alignment/ --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/*.params --fix_catres
python {scripts_dir}align_af2_with_inputs_and_copy_ligand.p

# **IV. FastMPNNdesign**

## Run FastMPNNdesign

In [11]:
################
### FastMPNN ###
################

### FILTERED & ALIGNED AF2 INPUT DIRECTORY ###
input_af2_aligned_str_dir = f'{af2_out_dir}predesigned_RFD2/filtered_structures/alignment/'
#num1: predesigned_RFD2
#num2: predesigned_inpainting

### OUTPUT DIRECTORY ###
output_directory = os.path.join(fastMPNNdesign_dir, 'RFD2/') #num1: RFD2, #num2: inpainting
os.makedirs(output_directory, exist_ok=True)

### INPUT LIGANDS ###
multiple_ligand_names = ["PSZ"]

### INPUT CST & PARAMS DIRECTORIES ###
cst_files_dir = f"{theozymes_dir}cst_files/"
params_and_theozyme_pdb_dir = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/"

### INPUT PARAMETERS ###
nstruct = 2
mpnn_flag = '--mpnn'
detect_pocket = '--detect_pocket'
bias_atoms = "H1"
bias_AAs = "ED"
bias_weight = 4
filter_flag = '' #--filter

### CONSTANTS ###
fastmpnn_script = f"{scripts_dir}enzyme_design/fastmpnndesign_ZnEsterase.py"
scoring_file = f"{scripts_dir}scoring/esterase_scoring.py"

# Generate commands
for ligand_name in multiple_ligand_names:
    # Find all PDB files for the current ligand
    pdb_files = glob.glob(os.path.join(input_af2_aligned_str_dir, f"*{ligand_name}*.pdb"))
    if len(pdb_files) == 0:
        continue

    # Find cst and params files, handling cases where they might not be found
    cst_files = glob.glob(os.path.join(cst_files_dir, f"*{ligand_name}*.cst"))
    if cst_files:
        cst_file = cst_files[0]
    else:
        print(f"No CST files found for {ligand_name}.")
        continue

    params_files = glob.glob(os.path.join(params_and_theozyme_pdb_dir, f"*{ligand_name}*.params"))
    if params_files:
        params_file = params_files[0]
    else:
        print(f"No PARAMS files found for {ligand_name}.")
        continue

    # Generate a command for each pdb file
    print ("Run the command below:")
    for pdb_file in pdb_files:
        command = (
            f"python {fastmpnn_script} --pdb {pdb_file} "
            f"--cstfile {cst_file} --params {params_file} --nstruct {nstruct} {mpnn_flag} {detect_pocket} "
            f"--bias_atoms {bias_atoms} --bias_AAs {bias_AAs} --position_bias {bias_weight} {filter_flag} --scoring {scoring_file} --outdir {output_directory}"
        )
        print (command)

Run the command below:
python {scripts_dir}enzyme_design/fastmpnndesign_ZnEsterase.py --pdb ./output/af2_out/predesigned_RFD2/filtered_structures/alignment/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ.pdb --cstfile ./input/theozymes/cst_files/PSZ_3H_NO_DEPROT_RES.cst --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/PSZ.params --nstruct 2 --mpnn --detect_pocket --bias_atoms H1 --bias_AAs ED --position_bias 4  --scoring {scripts_dir}scoring/esterase_scoring.py --outdir ./output/fastMPNNdesign/RFD2/
python {scripts_dir}enzyme_design/fastmpnndesign_ZnEsterase.py --pdb ./output/af2_out/predesigned_RFD2/filtered_structures/alignment/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_1_model_4_ptm_seed_0_unrelaxed_PSZ.pdb --cstfile ./input/theozymes/cst_files/PSZ_3H_NO_DEPROT_RES.cst --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/PSZ.params --nstruct 2 --mpnn --detect_pocket --bias_atoms H1 --bias_AAs ED --posit

## Analyze FastMPNNdesign

In [22]:
####################################################
### LOAD FASTMPNN SCORES AND GRAPH SUMMARY STATS ###
####################################################

### INPUTS ###
base_dir_for_fastmpnn = fastMPNNdesign_dir

subdirectories = ['RFD2', 'inpainting']
dataframe_names = [x+"_df" for x in subdirectories]

dataframes = {}
for subdir, df_name in zip(subdirectories, dataframe_names):
    found_files = glob.glob(os.path.join(base_dir_for_fastmpnn, subdir, "scores", "*.sc"))
    print (f'{len(found_files)} files were found!')
    if found_files:
        dfs = []
        for found_file in found_files:
            df = pd.read_csv(found_file, delim_whitespace=True, comment='#')
            dfs.append(df)
        dfs = pd.concat(dfs, ignore_index=True)
        dataframes[df_name] = dfs
        globals()[df_name] = dfs  # Assign DataFrame to a global variable
        print(f"DataFrame {df_name} loaded with {dfs.shape[0]} rows and {dfs.shape[1]} columns.")
        print('')
        dfs.head()

8 files were found!
DataFrame RFD2_df loaded with 160 rows and 44 columns.

22 files were found!
DataFrame inpainting_df loaded with 438 rows and 44 columns.



## Filter FastMPNNdesign structures 

In [23]:
##################################################################
### FILTER THE DATA FRAME BASED ON WHATEVER CRITERIA ARE IDEAL ###
##################################################################

show_df_head = False # True = show first 5 rows of df; False = do not 

dataframe_name = "RFD2_df"  # num1: RFD2_df, num2: inpainting_df
target_df = eval(dataframe_name)
labels_for_print = ['description', 'all_cst', 'H2O_hbond', 'oxy_hbond', 'cms_per_atom', 'corrected_ddg',\
                    'nlr_totrms', 'L_SASA', 'substrate_SASA', 'O2_hbond']

### MANDATORY (HARD-CODED) FILTERS ###
all_cst = 10 # 7
H2O_hbond = 0 # hydrogen bond with H of water molecule
O2_hbond = 1 # hydrogen bond with negatively charged oxygen

### OPTIONAL FILTERS ###
cms_per_atom = 6 # contact molecular surface
corrected_ddg = -20 # ddg
nlr_totrms = 1.0 # no-ligand-repack
score_per_res = 0.0 # usually always want < 0

### SASA UPPER + LOWER FILTERS ###
L_SASA_lower = 0 # ligand exposure
L_SASA_upper = 0.2 # ligand exposure
substrate_SASA_lower = 1 # substrate exposure
substrate_SASA_upper = 60 # substrate exposure


### FILTER ###
filters_P_Z = {
               ("L_SASA", "<="): L_SASA_upper, 
               ("L_SASA", ">="): L_SASA_lower, 
               ("substrate_SASA", "<="): substrate_SASA_upper, 
               ("substrate_SASA", ">="): substrate_SASA_lower, 
               ("all_cst", "<="): all_cst, 
               ("H2O_hbond", ">="): H2O_hbond, 
               ("cms_per_atom", ">="): cms_per_atom, 
               ("corrected_ddg", "<="): corrected_ddg, 
               ("nlr_totrms", "<="): nlr_totrms,
               ("score_per_res", "<"): score_per_res
              }

def filtering_df(dataframe_name, filters, target_df, print_stat=True):
    if print_stat: print (f'[{dataframe_name}] ({len(target_df)} designs)')
    
    converted_filters = []
    for key, condition in filters.items():
        key, operator = key
        converted_filter = f"target_df['{key}'] {operator} {condition}"
        converted_filters.append(converted_filter)
        if print_stat: 
            print(f'# [ {key.ljust(15)} {operator.ljust(2)} {str(condition).ljust(6)}]: {str(eval(converted_filter).to_list().count(True)).ljust(6)} ({eval(converted_filter).to_list().count(True)/len(target_df)*100:.1f}%)')
    
    filtered_df = target_df[eval(" & ".join(["("+x+")" for x in converted_filters]))]
    if print_stat:
        print(f'# [           ALL            ]: {len(filtered_df)} ({len(filtered_df)/len(target_df)*100:.1f}%)')
    return filtered_df


filtered_df = filtering_df(dataframe_name+"_P*Z", filters_P_Z, target_df)
filter_rate = float(len(filtered_df))/float(len(target_df))*100.0

print(f'{len(filtered_df)} of {len(target_df)} ({round(filter_rate,3)}%) passed')
print()

# Display the filtered DataFrame to verify the result
if show_df_head:
    n_descriptions = 40  # Adjust based on how many descriptions you want to print
    filtered_df.sort_values(by='all_cst', ascending=False, inplace=True)
    top_descriptions = (filtered_df['description']+".pdb").head(n_descriptions).tolist()
    print(' '.join(top_descriptions))
    print(filtered_df.head(10)[labels_for_print])

### SAVE DATAFRAMES ###
for subdir, df_name in zip(subdirectories, dataframe_names):
    if not df_name in dataframes:
        continue
    save_path = os.path.join(important_dfs_dir, f'{subdir}_fastmpnn_stats.csv')
    dataframes[df_name].to_csv(save_path, index=False)
    print(f"DataFrame {df_name} saved to {save_path}")

[RFD2_df_P*Z] (160 designs)
# [ L_SASA          <= 0.2   ]: 160    (100.0%)
# [ L_SASA          >= 0     ]: 160    (100.0%)
# [ substrate_SASA  <= 60    ]: 160    (100.0%)
# [ substrate_SASA  >= 1     ]: 82     (51.2%)
# [ all_cst         <= 10    ]: 139    (86.9%)
# [ H2O_hbond       >= 0     ]: 160    (100.0%)
# [ cms_per_atom    >= 6     ]: 160    (100.0%)
# [ corrected_ddg   <= -20   ]: 160    (100.0%)
# [ nlr_totrms      <= 1.0   ]: 159    (99.4%)
# [ score_per_res   <  0.0   ]: 160    (100.0%)
# [           ALL            ]: 67 (41.9%)
67 of 160 (41.875%) passed

DataFrame RFD2_df saved to ./working_dir/important_dfs/RFD2_fastmpnn_stats.csv
DataFrame inpainting_df saved to ./working_dir/important_dfs/inpainting_fastmpnn_stats.csv


In [18]:
#######################################################################
### COPY FILTERED STRUCTURES INTO SEPARATE DIRECTORY FOR ALPHAFOLD2 ###
#######################################################################

### INPUTS ###
dataframe_name = "filtered_df"
input_filtered_structures_dir = f'{fastMPNNdesign_dir}inpainting/'
#num 1: f'{fastMPNNdesign_dir}RFD2/'
#num 2: f'{fastMPNNdesign_dir}inpainting/'

output_filtered_structures_dir = os.path.join(input_filtered_structures_dir, "filtered_structures")
target_df = eval(dataframe_name)

### FUNCTION TO RUN ###s
def copy_filtered_fastmpnn_files(df, output_dir):
    existing_files = os.listdir(output_dir)
    print(f"Initial file count in the directory (Should = 0): {len(existing_files)}")
    if existing_files:
        print('')
        print(f"WARNING: The directory {output_dir} is not empty. It contains {len(existing_files)} files! Consider clearing that entire directory...")
        print('')

    descs_to_be_copied = df["description"].to_list()
    pdbs_to_be_copied = [os.path.join(input_filtered_structures_dir, x)+".pdb" for x in descs_to_be_copied]
    seqs_files = os.listdir(os.path.join(input_filtered_structures_dir, "seqs"))
    for seqs_file in seqs_files:
        if seqs_file[:seqs_file.index("_T0.")] in descs_to_be_copied:
            pdbs_to_be_copied.append(os.path.join(input_filtered_structures_dir, "seqs", seqs_file))
    
    print(f"Number of PDBs to Copy: {len(pdbs_to_be_copied)}")
    print(f"Copying Files to the Directory: {output_dir}")
    
    count = 0
    start_time = time.time()
    error_files = []

    for pdb_path in pdbs_to_be_copied:

        try:
            shutil.copy(pdb_path, output_dir)
            count += 1

            if np.log10(count) % 1 == 0:
                current_time = time.ctime()
                elapsed_time = time.time() - start_time
                files_left = (len(df)*2 - count)
                average_time_per_file = elapsed_time / count
                estimated_time_remaining = max(0, average_time_per_file * files_left)
                print(f"[{current_time}] {count} files copied successfully. Elapsed time: {elapsed_time:.2f} seconds. Average time per file: {average_time_per_file:.4f} seconds. Estimated time remaining: {estimated_time_remaining:.2f} seconds.")

        except Exception as e:
            error_files.append((pdb_path, str(e)))

    print(f"Total files copied: {count}")
    if error_files:
        print("Failed to copy the following files:")
        for error in error_files:
            print(error)

### RUN THE SCRIPT ###
os.makedirs(output_filtered_structures_dir, exist_ok=True)
copy_filtered_fastmpnn_files(target_df, output_filtered_structures_dir)
print('')
print('Done!')

Initial file count in the directory (Should = 0): 0
Number of PDBs to Copy: 3168
Copying Files to the Directory: ./output/fastMPNNdesign/inpainting/filtered_structures
[Wed Apr 23 12:04:29 2025] 1 files copied successfully. Elapsed time: 0.02 seconds. Average time per file: 0.0190 seconds. Estimated time remaining: 10.92 seconds.
[Wed Apr 23 12:04:29 2025] 10 files copied successfully. Elapsed time: 0.09 seconds. Average time per file: 0.0091 seconds. Estimated time remaining: 5.16 seconds.
[Wed Apr 23 12:04:29 2025] 100 files copied successfully. Elapsed time: 0.93 seconds. Average time per file: 0.0093 seconds. Estimated time remaining: 4.44 seconds.
[Wed Apr 23 12:04:38 2025] 1000 files copied successfully. Elapsed time: 9.68 seconds. Average time per file: 0.0097 seconds. Estimated time remaining: 0.00 seconds.
Total files copied: 3168

Done!


# **V. AlphaFold2 (2nd Iteration)**

## Run AF2

In [20]:
#######################################
### predict designs with AlphaFold2 ###
#######################################

### CONSTANTS ###
apptainer = './containers/crispy.sif' 
superfold = '{software_dir}superfold/run_superfold.py'

### CHANGE THESE EVERY TIME ###
af2_inputs_directory = f'{fastMPNNdesign_dir}RFD2/filtered_structures/'
#num 1: f'{fastMPNNdesign_dir}RFD2/filtered_structures/'
#num 2: f'{fastMPNNdesign_dir}inpainting/filtered_structures/'

af2_output_dir = f'{af2_out_dir}predesigned_RFD2_i2/'
#num 1: f'{af2_out_dir}predesigned_RFD2_i2/'
#num 2: f'{af2_out_dir}predesigned_inpainting_i2/'
os.makedirs(af2_output_dir, exist_ok=True)

### ALPHAFOLD PARAMETERS ###
model_number = 4 # 4 is recommended, 5 is probably similar
model_type = 'monomer_ptm'
version = 'monomer'

nstruct = 1 # how many structures to generate as outputs, default = 1
num_recycles = 6 # default = 3
num_ensemble = 2 # default in code is 1 but CASP competition used 8
mock_msa_depth = 1 # default is 1, but higher might = better ? goes to +inf
recycle_tol = 0.0 # default is 0.0, what convergence rmsd to stop at after recycling

pct_seq_mask = 0.15 # 0.15 is default, percent of sequence to make during inference
output_summary = '' #'--output_summary' # ENABLE IF YES, otherwise ''
output_pae = '--output_pae' # ENABLE IF YES, otherwise ''
amber_relax = '' #'--amber_relax' # ENABLE IF YES, otherwise ''
initial_guess = '' #'--initial_guess {pdb}' # ENABLE IF YES, otherwise ''

### MAKE THE FUNCTION ###
cmd = f'{apptainer} {superfold} --out_dir {af2_output_dir} --model {model_number} --type {model_type} --version {version} \
--nstruct {nstruct} --max_recycles {num_recycles} --num_ensemble {num_ensemble} --mock_msa_depth {mock_msa_depth} --recycle_tol {recycle_tol} \
--pct_seq_mask {pct_seq_mask} {output_summary} {output_pae} {amber_relax}' #{initial_guess}'

for i, pdb in enumerate(glob.glob(os.path.join(af2_inputs_directory, f"*.pdb"))): 
    if i == 0 or i % 5 != 0:
        cmd += f" {pdb}"
    else:
        print(cmd)
        cmd = f'{apptainer} {superfold} --out_dir {af2_output_dir} --model {model_number} \
        --type {model_type} --version {version} \
        --nstruct {nstruct} --max_recycles {num_recycles} --num_ensemble {num_ensemble} \
        --mock_msa_depth {mock_msa_depth} --recycle_tol {recycle_tol} \
        --pct_seq_mask {pct_seq_mask} {output_summary} {output_pae} {amber_relax} \
        {pdb}' # {initial_guess} {pdb}
print(cmd)

./containers/crispy.sif {software_dir}superfold/run_superfold.py --out_dir ./output/af2_out/predesigned_RFD2_i2/ --model 4 --type monomer_ptm --version monomer --nstruct 1 --max_recycles 6 --num_ensemble 2 --mock_msa_depth 1 --recycle_tol 0.0 --pct_seq_mask 0.15  --output_pae  ./output/fastMPNNdesign/RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_0.pdb ./output/fastMPNNdesign/RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_0_T0.1_0.pdb ./output/fastMPNNdesign/RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_0_T0.1_1.pdb ./output/fastMPNNdesign/RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_0_T0.1_2.pdb ./output/fastMPNNdesign/RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_

## AF2 Analysis

In [21]:
##################################################################
### PARSE & PROCESS AF2 OUTPUT FILES ONLY NEED TO DO THIS ONCE ###
##################################################################

# INPUT DIRECTORY TO PARSE
af2_dir = f'{af2_out_dir}predesigned_RFD2_i2/'
#num 1: f'{af2_out_dir}predesigned_RFD2_i2/'
#num 2: f'{af2_out_dir}predesigned_inpainting_i2/'

# Constants
script = f"{scripts_dir}af2_parse_processing_multi.py"
optional_cpus = 19  # Optional: specify the number of CPUs to use, or remove this variable to use default logic

# Make command
command = f"python {script} {af2_dir}"
if 'optional_cpus' in locals():
    command += f" --cpus {optional_cpus}"

# Print submission
print("SUBMIT THIS:")
print(command)
print('')

# Print output .csv path
print("OUTPUT CSV FILE HERE:")
print(af2_dir + 'af2_out_parsed.csv')

SUBMIT THIS:
python {scripts_dir}af2_parse_processing_multi.py ./output/af2_out/predesigned_RFD2_i2/ --cpus 19

OUTPUT CSV FILE HERE:
./output/af2_out/predesigned_RFD2_i2/af2_out_parsed.csv


## Filter AF2 structures based on Global Metric

In [22]:
############################################
### LOAD AND FILTER AF2 BY SUMMARY STATS ###
############################################

# Correct the path to where your actual CSV file is located
csv_file_path = f'{af2_out_dir}predesigned_RFD2_i2/af2_out_parsed.csv'
#num 1: f'{af2_out_dir}predesigned_RFD2_i2/'
#num 2: f'{af2_out_dir}predesigned_inpainting_i2/'

output_filtered_structures_dir = f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/'
#num 1: f'{af2_out_dir}predesigned_RFD2/filtered_structures_i2/'
#num 2: f'{af2_out_dir}predesigned_inpainting/filtered_structures_i2/'

# Select parameters for filtering -> EXTREMELY HARSH IN THIS CASE
plddt_cutoff = 60 # in pLDDT units -> usually 70-80
rmsd_cutoff = 3 # Å, I think this is Cα (1.5 is typical, 2.0 might be a stretch but okay for big structures)
pae_cutoff = 100 # below 5 is harsh but good
pTM_cutoff = 0.5 # Supposedly above 0.75 is good
convergence_cutoff = 1.5

# Load the CSV file into a DataFrame
af2_scores_df = pd.read_csv(csv_file_path)

af2_mask = ((af2_scores_df['mean_plddt'] >= plddt_cutoff) & 
            (af2_scores_df['rmsd_to_input'] <= rmsd_cutoff) &
            (af2_scores_df['mean_pae'] <= pae_cutoff) & 
            (af2_scores_df['pTMscore'] >= pTM_cutoff) &
            (af2_scores_df['af2_convergence_tol'] <= convergence_cutoff)
           )

filtered_af2_scores_df = af2_scores_df[af2_mask]
af2_filter_rate = float(len(filtered_af2_scores_df))/float(len(af2_scores_df))*100.0
print(f'{len(filtered_af2_scores_df)} of {len(af2_scores_df)} ({round(af2_filter_rate,3)}%) with plddt >= {plddt_cutoff} and Cα rmsd <={rmsd_cutoff}')

### FUNCTION TO RUN ###
def copy_files(df, pdb_col, json_col, output_dir):
    existing_files = os.listdir(output_dir)
    print(f"Initial file count in the directory (Should = 0): {len(existing_files)}")

    count = 0
    start_time = time.time()
    error_files = []

    for index, row in df.iterrows():
        pdb_path = row[pdb_col]
        json_path = row[json_col]

        try:
            shutil.copy(pdb_path, output_dir)
            shutil.copy(json_path, output_dir)
            count += 2  # Increment for each PDB and JSON file copied

            if count % 2000 == 0:
                current_time = time.ctime()
                elapsed_time = time.time() - start_time
                files_left = (len(df)*2 - count)
                average_time_per_file = elapsed_time / count
                estimated_time_remaining = max(0, average_time_per_file * files_left)
                print(f"[{current_time}] {count} files copied successfully. Elapsed time: {elapsed_time:.2f} seconds. Average time per file: {average_time_per_file:.4f} seconds. Estimated time remaining: {estimated_time_remaining:.2f} seconds.")

        except Exception as e:
            error_files.append((pdb_path, json_path, str(e)))

    print(f"Total files copied: {count}")
    if error_files:
        print("Failed to copy the following files:")
        for error in error_files:
            print(error)

### RUN THE SCRIPT ###
os.makedirs(output_filtered_structures_dir, exist_ok=True)
copy_files(filtered_af2_scores_df, 'pdb_path', 'af2_json_path', output_filtered_structures_dir)
print('')
print('Done!')

21 of 51 (41.176%) with plddt >= 60 and Cα rmsd <=3
Initial file count in the directory (Should = 0): 0
Total files copied: 42

Done!


## Analyze AF2 on Side Chain Metrics

### Run Both of These Once to Generate Score Files

In [23]:
##################################################################
### PARSE & PROCESS AF2 OUTPUT FILES ONLY NEED TO DO THIS ONCE ###
##################################################################

# INPUT DIRECTORY TO PARSE
af2_directory_to_parse = f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/'
#num 1: f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/'
#num 2: f'{af2_out_dir}predesigned_inpainting_i2/filtered_structures/'

# Constants
script = f"{scripts_dir}af2_parse_processing_multi.py"
optional_cpus = 19  # Optional: specify the number of CPUs to use, or remove this variable to use default logic
optional_basename_for_summary_stats = "scores.sc"

# Make command
command = f"python {script} {af2_directory_to_parse}"
if 'optional_cpus' in locals():
    command += f" --cpus {optional_cpus}"
if 'optional_basename_for_summary_stats' in locals():
    command += f" --optional_basename_for_summary_stats {optional_basename_for_summary_stats}"

# Print submission
print("SUBMIT THIS:")
print(command)
print('')

# Print output .csv path
print("OUTPUT CSV FILE HERE:")
print(af2_directory_to_parse + f'{optional_basename_for_summary_stats}')

SUBMIT THIS:
python {scripts_dir}af2_parse_processing_multi.py ./output/af2_out/predesigned_RFD2_i2/filtered_structures/ --cpus 19 --optional_basename_for_summary_stats scores.sc

OUTPUT CSV FILE HERE:
./output/af2_out/predesigned_RFD2_i2/filtered_structures/scores.sc


In [24]:
##################################################################################################
### CALCULATE SIDE CHAIN RMSDS & OTHER RMSDS FOR CATALYTIC RESIDUES IN FILTERED STRUCTURES DIR ###
##################################################################################################

### INPUT SCORE PATH + OUTPUT SCORE PATH & REFERENCE FILES ###
score_path = os.path.join(f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures', 'scores.sc')
output_file = os.path.join(f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures', 'updated_scores.sc')
#num 1: f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures'
#num 2: f'{af2_out_dir}predesigned_inpainting_i2/filtered_structures'

ref_pdbs_dir = os.path.join(f'{fastMPNNdesign_dir}RFD2', 'filtered_structures/')
#num 1: f'{fastMPNNdesign_dir}RFD2/filtered_structures/'
#num 2: f'{fastMPNNdesign_dir}inpainting/filtered_structures/'

### INPUT PARAMS ###
params_path = os.path.join(invrot_dir, "COMBINED_PHENYLACETATE_HHH", "COMBINED_PARAMS", "*.params")  # Ensure params include .pdb if needed

### INPUT ATOM RMSDS OF CAT RESIDUES TO CALCULATE & NUM OF SUFFIXES ###
atom_groups_json = { #MANUALLY INPUT POSSIBLE CATALYTIC RESIDUES!!
    "HIS": [
        {"atoms": ["N", "CA", "C", "O"], "label": "bb_rmsd"},
        {"atoms": ["N", "CA", "C", "O", "CB"], "label": "cb_bb_rmsd"},
        {"atoms": ["CG", "CE1", "ND1", "NE2", "CD2"], "label": "imidazole_rmsd"},
        {"atoms": ["CA", "CB"], "label": "ca_cb_rmsd"},
        {"atoms": ["ND1", "NE2"], "label": "imid_sc_nitrogen_rmsd"}
    ]
}

max_possible_num_of_suffixes = 1
column_name_for_pdb_path = 'pdb_path'

### CONSTANTS ###
script = '{scripts_dir}sidechain_rmsd_and_info_af2_matching_res.py'
atom_groups = json.dumps(atom_groups_json)

# Build the command
command = (
    f'python {script} --scorefile {score_path} --ref_pdbs_dir {ref_pdbs_dir} --params {params_path} --output_file {output_file} '
    f'--max_possible_num_of_suffixes {max_possible_num_of_suffixes} --atom_groups \'{atom_groups}\''
)
print("Command to execute:")
print(command)
print('')
print('New scorefile:')
print(f'{output_file}')

Command to execute:
python {scripts_dir}sidechain_rmsd_and_info_af2_matching_res.py --scorefile ./output/af2_out/predesigned_RFD2_i2/filtered_structures/scores.sc --ref_pdbs_dir ./output/fastMPNNdesign/RFD2/filtered_structures/ --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/*.params --output_file ./output/af2_out/predesigned_RFD2_i2/filtered_structures/updated_scores.sc --max_possible_num_of_suffixes 1 --atom_groups '{"HIS": [{"atoms": ["N", "CA", "C", "O"], "label": "bb_rmsd"}, {"atoms": ["N", "CA", "C", "O", "CB"], "label": "cb_bb_rmsd"}, {"atoms": ["CG", "CE1", "ND1", "NE2", "CD2"], "label": "imidazole_rmsd"}, {"atoms": ["CA", "CB"], "label": "ca_cb_rmsd"}, {"atoms": ["ND1", "NE2"], "label": "imid_sc_nitrogen_rmsd"}]}'

New scorefile:
./output/af2_out/predesigned_RFD2_i2/filtered_structures/updated_scores.sc


### Do a Light Side Chain Filter Before Global Alignment & Harsh SC Filter

In [25]:
##################################################################################
### COMPUTE SIDE CHAIN STATISTICS FROM SCORES DF & DO GENTLE/LOGICAL FILTERING ###
##################################################################################

### INPUT PATH ###
scores_filepath = f"{af2_out_dir}predesigned_RFD2_i2/filtered_structures/updated_scores.sc"
inpaint_or_diffusion = "RFD2" #RFD2 or Inpainting
#num 1: f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/updated_scores.sc'
#num 2: f'{af2_out_dir}predesigned_inpainting_i2/filtered_structures/updated_scores.sc'

### INPUT SPECIFY COLUMN GROUPS FOR CALCULATION ###
column_groups = {
    'cat_HIS_bb': ['cat_HIS1_bb_rmsd', 'cat_HIS2_bb_rmsd', 'cat_HIS3_bb_rmsd'],
    'cat_HIS_cb_bb': ['cat_HIS1_cb_bb_rmsd', 'cat_HIS2_cb_bb_rmsd', 'cat_HIS3_cb_bb_rmsd'],
    'cat_HIS_ca_cb_ONLY': ['cat_HIS1_ca_cb_rmsd', 'cat_HIS2_ca_cb_rmsd', 'cat_HIS3_ca_cb_rmsd'],
    'cat_HIS_imidazole': ['cat_HIS1_imidazole_rmsd', 'cat_HIS2_imidazole_rmsd', 'cat_HIS3_imidazole_rmsd'],
    'cat_HIS_imidazole_tip_Ns': ['cat_HIS1_imid_sc_nitrogen_rmsd', 'cat_HIS2_imid_sc_nitrogen_rmsd', 'cat_HIS3_imid_sc_nitrogen_rmsd']
}

### FUNCTION TO CALCULATE MEAN RMSD AND RMS OF RMSDs FOR SPECIFIED COLUMN GROUPS ###
def calculate_group_rmsd(df, column_groups):
    for group_name, columns in column_groups.items():
        mean_col_name = f"{group_name}_mean_rmsd"
        rms_col_name = f"{group_name}_rms_of_rmsds"
        df[mean_col_name] = df[columns].mean(axis=1)
        df[rms_col_name] = np.sqrt((df[columns]**2).mean(axis=1))

### EXECUTE PLOTTING AND PRINTING ###
sidechain_rmsd_df = pd.read_csv(scores_filepath, sep=',', na_values='')
calculate_group_rmsd(sidechain_rmsd_df, column_groups)

In [26]:
##################################################################################
### COMPUTE SIDE CHAIN STATISTICS FROM SCORES DF & DO GENTLE/LOGICAL FILTERING ###
##################################################################################

### NEW FILTER FROM SIDE CHAIN 2 ###
cat_HIS_bb_rms_of_rmsds_cutoff = 0.85 #2 # Å
cat_HIS_cb_bb_rms_of_rmsds_cutoff = 0.85 # Å
cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff = 0.85 # Å
cat_HIS_imidazole_rms_of_rmsds_cutoff = 1.25 # Å
cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff = 1.25 # Å

# Individual filter calculations with specific naming
filter_masks = {
    'cat_HIS_bb_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_bb_rms_of_rmsds'] <= cat_HIS_bb_rms_of_rmsds_cutoff,
    'cat_HIS_cb_bb_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_cb_bb_rms_of_rmsds'] <= cat_HIS_cb_bb_rms_of_rmsds_cutoff,
    'cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_ca_cb_ONLY_rms_of_rmsds'] <= cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff,
    'cat_HIS_imidazole_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_imidazole_rms_of_rmsds'] <= cat_HIS_imidazole_rms_of_rmsds_cutoff,
    'cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask': sidechain_rmsd_df['cat_HIS_imidazole_tip_Ns_rms_of_rmsds'] <= cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff
}

# Calculate pass counts and rates for each filter
filter_results = {filter_name: (mask.sum(), len(sidechain_rmsd_df), mask.mean() * 100) for filter_name, mask in filter_masks.items()}

# Overall filtering result using OR logic
cumulative_filter_mask_or = filter_masks['cat_HIS_bb_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_cb_bb_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_imidazole_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask']

cumulative_pass_count_or = cumulative_filter_mask_or.sum()
cumulative_total_or = len(sidechain_rmsd_df)
cumulative_pass_rate_or = cumulative_pass_count_or / cumulative_total_or * 100

### DO THE PRINT STATEMENT (FANCY) ###
print('########################## SIDE CHAIN RMSD METRICS FILTER ##########################')
for filter_name, (pass_count, total, pass_rate) in filter_results.items():
    specific_criteria = filter_name.replace('_mask', '').replace('_', ' ').replace('cat', 'catalytic').upper()
    cutoff_value = {
        'cat_HIS_bb_rms_of_rmsds_mask': cat_HIS_bb_rms_of_rmsds_cutoff,
        'cat_HIS_cb_bb_rms_of_rmsds_mask': cat_HIS_cb_bb_rms_of_rmsds_cutoff,
        'cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask': cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff,
        'cat_HIS_imidazole_rms_of_rmsds_mask': cat_HIS_imidazole_rms_of_rmsds_cutoff,
        'cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask': cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff
    }[filter_name]
    print(f'{pass_count} of {total} ({pass_rate:.3f}%) have {specific_criteria} <= {cutoff_value}')

# Maintaining the cumulative print statement from the provided script
print('')
print(f'{cumulative_pass_count_or} of {cumulative_total_or} ({cumulative_pass_rate_or:.3f}%) pass at least one filter!')
print('####################################################################################')

### MAKE THE FINAL DATA FRAME ###
final_filtered_df = sidechain_rmsd_df[cumulative_filter_mask_or]

### PRINT OUT COLUMN NAMES USED IN FILTERING ###
filtered_columns = ', '.join([key.replace('_mask', '') for key in filter_masks.keys()])
print(f'Columns used in filtering: {filtered_columns}')

########################## SIDE CHAIN RMSD METRICS FILTER ##########################
15 of 21 (71.429%) have CATALYTIC HIS BB RMS OF RMSDS <= 0.85
14 of 21 (66.667%) have CATALYTIC HIS CB BB RMS OF RMSDS <= 0.85
14 of 21 (66.667%) have CATALYTIC HIS CA CB ONLY RMS OF RMSDS <= 0.85
0 of 21 (0.000%) have CATALYTIC HIS IMIDAZOLE RMS OF RMSDS <= 1.25
0 of 21 (0.000%) have CATALYTIC HIS IMIDAZOLE TIP NS RMS OF RMSDS <= 1.25

16 of 21 (76.190%) pass at least one filter!
####################################################################################
Columns used in filtering: cat_HIS_bb_rms_of_rmsds, cat_HIS_cb_bb_rms_of_rmsds, cat_HIS_ca_cb_ONLY_rms_of_rmsds, cat_HIS_imidazole_rms_of_rmsds, cat_HIS_imidazole_tip_Ns_rms_of_rmsds


## Apo Structure Relax

In [28]:
#######################################################
### RUN PYFAST RELAX W A BUNCH OF STAT CALCULATIONS ### 
### (MOSTLY JUST SIDE CHAIN PACKING)                ###  
#######################################################

### GENERATION OF RELAX (USUALLY ONLY INPUT) ###

# Regulars, do not change much
script = '{scripts_dir}pyfastrelax_and_hbond_APO.py'

# Change Only if Necessary | INPUT DIR #mpnn_out_dir & packed/---> (OLD GENERATION, ASSUMES PREVIOUS Gen)
input_directory_pyfast_relax = f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/'
#num1: f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/' 
#num2: f'{af2_out_dir}predesigned_inpainting_i2/filtered_structures/' 

# Change Only if Necessary | OUTPUT RELAX DIRECTORY
pyfast_relax_out_directory = f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/APO_RELAXED/'
#num1: f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/APO_RELAXED/'
#num2: f'{af2_out_dir}predesigned_inpainting_i2/filtered_structures/APO_RELAXED/'
os.makedirs(pyfast_relax_out_directory, exist_ok=True)

# Change Only if Necessary | Generation Format Should Update Accordingly
for pdb in glob.glob(f'{input_directory_pyfast_relax}*pdb'):  
    cmd_txt = (f'python {script} '
               f'--pdb {pdb} '
               f'--out_dir {pyfast_relax_out_directory}')
    print (cmd_txt) 


python {scripts_dir}pyfastrelax_and_hbond_APO.py --pdb ./output/af2_out/predesigned_RFD2_i2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_1_model_4_ptm_seed_0_unrelaxed.pdb --out_dir ./output/af2_out/predesigned_RFD2_i2/filtered_structures/APO_RELAXED/
python {scripts_dir}pyfastrelax_and_hbond_APO.py --pdb ./output/af2_out/predesigned_RFD2_i2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_1_T0.2_3_model_4_ptm_seed_0_unrelaxed.pdb --out_dir ./output/af2_out/predesigned_RFD2_i2/filtered_structures/APO_RELAXED/
python {scripts_dir}pyfastrelax_and_hbond_APO.py --pdb ./output/af2_out/predesigned_RFD2_i2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_8_T0.1_0_model_4_ptm_seed_0_unrelaxed.pdb --out_dir ./output/af2_out/predesigned_RFD2_i2/filtered_structures/APO_RELAXED/
python {scripts_dir}pyfastrelax_and

In [ ]:
###############################
### ANALYZE RELAXED OUTPUTS ###
###############################

# Define the constant variables
script_path = f"{scripts_dir}pyfastrelax_stat_processingAPO.py"

# CHANGE EVERY TIME 
part_diffusion_relax_out_directory_path = f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/APO_RELAXED/'
#num1: f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/APO_RELAXED/'
#num2: f'{af2_out_dir}predesigned_inpainting_i2/filtered_structures/APO_RELAXED/'
number_of_cores = '--nproc 10'

# Format the command string
command = f"python {script_path} {part_diffusion_relax_out_directory_path} {number_of_cores}"

# Print the command
print('COMMAND:')
print(command)
print('')
print('OUTPUT FILE:')
print(f'{part_diffusion_relax_out_directory_path}parsed_energy_data.csv')

COMMAND:
python {scripts_dir}pyfastrelax_stat_processingAPO.py ./output/af2_out/predesigned_RFD2_i2/filtered_structures/APO_RELAXED/ --nproc 300

OUTPUT FILE:
./output/af2_out/predesigned_RFD2_i2/filtered_structures/APO_RELAXED/parsed_energy_data.csv


In [30]:
#############################################################
### FILTER THE DATA FRAME BASED ON THE CRITERIA ARE IDEAL ###
#############################################################

# Dynamically create file path and DataFrame names using the generation number
file_path = f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/APO_RELAXED/parsed_energy_data.csv'
#num 1: f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/APO_RELAXED/parsed_energy_data.csv'
#num 2: f'{af2_out_dir}predesigned_inpainting_i2/filtered_structures/APO_RELAXED/parsed_energy_data.csv'

relaxed_stats_APO_df = pd.read_csv(file_path)

### FILTER THE DATA FRAME ###
filtered_relaxed_stats_APO_df = relaxed_stats_APO_df[
    ((relaxed_stats_APO_df['sap_score'] <= 75) |
    (relaxed_stats_APO_df['net_charge_in_design_not_w_his'].abs() >= 20)) &
    (relaxed_stats_APO_df['hydrophobic_exposure_sasa_in_design'] <= 1500)
]

# Print the count of rows in the filtered DataFrame and the total in the original DataFrame
print(f"Rows matching criteria: {filtered_relaxed_stats_APO_df.shape[0]} out of {relaxed_stats_APO_df.shape[0]} total rows")

Rows matching criteria: 13 out of 13 total rows


## Copy Ligand Coords into Structure

In [31]:
##############################################
### GENERATE COMMANDS FOR ALIGNING LIGANDS ###
##############################################

import time

### INPUTS ###
filtered_af2_sc_AND_global_scores_df = filtered_relaxed_stats_APO_df # INPUT DF
column_of_pdb_paths = 'pdb_path' # COLUMN OF DF W/PDB PATHS

ref_pdbs_dir = os.path.join(fastMPNNdesign_dir, 'RFD2', 'filtered_structures/')
#num 1: RFD2
#num 2: inpainting

params_path = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/"

align_af2_out_directory = os.path.join(f"{af2_out_dir}", "predesigned_RFD2_i2", "filtered_structures", "alignment/")
# num1: predesigned_RFD2_i2
# num2: predesigned_inpainting_i2

### CONSTANTS ###
align_script = f"{scripts_dir}align_af2_with_inputs_and_copy_ligand.py"

### REFERENCE PDB PROCESSING ###
ref_pdb_files = glob.glob(os.path.join(ref_pdbs_dir, '*.pdb'))
ref_pdb_map = {os.path.basename(p).replace('.pdb', ''): p for p in ref_pdb_files}
ref_pdb_usage = Counter()

### AUTOMATED COMMAND WRITING & REFERENCE MATCHING ###
os.makedirs(align_af2_out_directory, exist_ok=True)
max_usages_per_ref_pdb = 30  # Number of times a reference pdb can be used

total_rows = len(filtered_af2_sc_AND_global_scores_df)
rows_processed = 0
update_interval = 1000

start_time = time.time()
cmds = []
for index, row in filtered_af2_sc_AND_global_scores_df.iterrows():
    pdb_path = row[column_of_pdb_paths]
    pdb_name = os.path.basename(pdb_path).split('.')[0]

    # Find matching reference PDB
    matching_ref = None
    for ref_name, ref_path in ref_pdb_map.items():
        if pdb_name.startswith(ref_name) and ref_pdb_usage[ref_name] < max_usages_per_ref_pdb:
            matching_ref = ref_path
            ref_pdb_usage[ref_name] += 1
            if ref_pdb_usage[ref_name] >= max_usages_per_ref_pdb:
                del ref_pdb_map[ref_name]  # Remove from future considerations
            break

    if matching_ref:
        command_align = (f"python {align_script} "
                         f"--pdb {pdb_path} "
                         f"--ref {matching_ref} "
                         f"--outdir {align_af2_out_directory} "
                         f"--params {os.path.join(params_path, '*.params')} "
                         f"--fix_catres")
        cmds.append(command_align)

    rows_processed += 1
    if rows_processed % update_interval == 0 or rows_processed == total_rows:
        elapsed_time = time.time() - start_time
        time_per_row = elapsed_time / rows_processed
        estimated_time_remaining = time_per_row * (total_rows - rows_processed)
        print(f"{rows_processed} rows processed, {total_rows - rows_processed} remaining. "
              f"{len(ref_pdb_map)} ref pdb files available for matching. "
              f"Elapsed time: {elapsed_time:.2f} s. Estimated time left: {estimated_time_remaining:.2f} s.")
print('')
print("Run below commands:")
print("\n".join(cmds))

13 rows processed, 0 remaining. 737 ref pdb files available for matching. Elapsed time: 0.00 s. Estimated time left: 0.00 s.

Run below commands:
python {scripts_dir}align_af2_with_inputs_and_copy_ligand.py --pdb ./output/af2_out/predesigned_RFD2_i2/filtered_structures/APO_RELAXED/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_1_model_4_ptm_seed_0_unrelaxed_relaxedAPO.pdb --ref ./output/fastMPNNdesign/RFD2/filtered_structures/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_1.pdb --outdir ./output/af2_out/predesigned_RFD2_i2/filtered_structures/alignment/ --params ./input/invrot/COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/*.params --fix_catres
python {scripts_dir}align_af2_with_inputs_and_copy_ligand.py --pdb ./output/af2_out/predesigned_RFD2_i2/filtered_structures/APO_RELAXED/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_1_T0.2_3_model_4_ptm_seed_0_unrelaxed_r

## Sequence Clustering & Filtering Down

In [32]:
##########################################################################################################################
### DO FILTERING BY SIDECHAINS & GLOBAL METRICS (DAMAGE CONTROL IN THIS CASE) AND SEQUENCE CLUSTER & FILTER DOWN TOP 3 ###
##########################################################################################################################

### INPUTS ###
af2_file = f"{af2_out_dir}predesigned_RFD2_i2/filtered_structures/updated_scores.sc"
# num1: predesigned_RFD2_i2
# num2: predesigned_inpainting_i2
apo_relax_file = f'{af2_out_dir}predesigned_RFD2_i2/filtered_structures/APO_RELAXED/parsed_energy_data.csv'
# num1: predesigned_RFD2_i2
# num2: predesigned_inpainting_i2

# Specify column groups for calculation
column_groups = {
    'cat_HIS_bb': ['cat_HIS1_bb_rmsd', 'cat_HIS2_bb_rmsd', 'cat_HIS3_bb_rmsd'],
    'cat_HIS_cb_bb': ['cat_HIS1_cb_bb_rmsd', 'cat_HIS2_cb_bb_rmsd', 'cat_HIS3_cb_bb_rmsd'],
    'cat_HIS_ca_cb_ONLY': ['cat_HIS1_ca_cb_rmsd', 'cat_HIS2_ca_cb_rmsd', 'cat_HIS3_ca_cb_rmsd'],
    'cat_HIS_imidazole': ['cat_HIS1_imidazole_rmsd', 'cat_HIS2_imidazole_rmsd', 'cat_HIS3_imidazole_rmsd'],
    'cat_HIS_imidazole_tip_Ns': ['cat_HIS1_imid_sc_nitrogen_rmsd', 'cat_HIS2_imid_sc_nitrogen_rmsd', 'cat_HIS3_imid_sc_nitrogen_rmsd']
}

### FUNCTION TO CALCULATE MEAN RMSD AND RMS OF RMSDs FOR SPECIFIED COLUMN GROUPS ###
def calculate_group_rmsd(df, column_groups):
    for group_name, columns in column_groups.items():
        rms_col_name = f"{group_name}_rms_of_rmsds"
        df[rms_col_name] = np.sqrt((df[columns]**2).mean(axis=1))

### FUNCTION TO EXTRACT BASE NAME FOR MATCHING ###
def extract_base_name_for_matching(pdb_path):
    base_name = os.path.basename(pdb_path)
    parts = base_name.split('_model_4_ptm_seed_0_unrelaxed')
    if len(parts) > 2:
        return '_model_4_ptm_seed_0_unrelaxed'.join(parts[:2])
    return base_name

### FUNCTION TO EXTRACT BASE NAME ###
def extract_base_name(pdb_path):
    base_name = os.path.basename(pdb_path)
    if '_T0' in base_name:
        return base_name.split('_T0')[0]
    elif '_model_4_ptm_seed_0_unrelaxed_relaxedAPO_' in base_name:
        return base_name.split('_model_4_ptm_seed_0_unrelaxed_relaxedAPO_')[0]
    else:
        return base_name

### LOAD DATA FRAMES ###
# Load AF2 DataFrame
af2_df = pd.read_csv(af2_file, sep=',', na_values='')
calculate_group_rmsd(af2_df, column_groups)
af2_df = af2_df.add_suffix('_af2_out')
af2_df['af2_names_for_matching'] = af2_df['pdb_path_af2_out'].apply(extract_base_name_for_matching)

# Load APO Relax DataFrame
apo_relax_df = pd.read_csv(apo_relax_file)
apo_relax_df = apo_relax_df.add_suffix('_af2_apo_relax_out')
apo_relax_df['af2_names_for_matching'] = apo_relax_df['pdb_path_af2_apo_relax_out'].apply(extract_base_name_for_matching)

# Print the shape of each DataFrame before merging
print("AF2 DataFrame before merge:")
print(af2_df.shape)
print("\nAPO Relax DataFrame before merge:")
print(apo_relax_df.shape)

### MERGE DATA FRAMES ###
# First merge af2_df and apo_relax_df
merged_df = pd.merge(af2_df, apo_relax_df, on='af2_names_for_matching', suffixes=('_af2', '_apo_relax'))

# Print the shape of the merged DataFrame
print("\nMerged DataFrame:")
print(merged_df.shape)

### CREATE SECOND SHELL SEQ CLUSTERING COLUMN ###
merged_df['second_shell_seq_clustering'] = merged_df['pdb_path_af2_out'].apply(extract_base_name)

### GROUP BY SECOND SHELL SEQ CLUSTERING ###
grouped = merged_df.groupby('second_shell_seq_clustering')

# Get the number of unique groups
num_unique_groups = grouped.ngroups

# Count the number of rows in each group
group_counts = grouped.size()

# Count how many groups have more than 3, 5, and 7 rows
groups_more_than_3 = group_counts[group_counts > 3].count()
groups_more_than_5 = group_counts[group_counts > 5].count()
groups_more_than_7 = group_counts[group_counts > 7].count()

### PRINT RESULTS ###
print(f'\nTotal number of rows in the merged DataFrame: {len(merged_df)}')
print(f'Number of unique groups: {num_unique_groups}')
print(f'Number of groups with more than 3 rows: {groups_more_than_3} (Total rows: {group_counts[group_counts > 3].sum()})')
print(f'Number of groups with more than 5 rows: {groups_more_than_5} (Total rows: {group_counts[group_counts > 5].sum()})')
print(f'Number of groups with more than 7 rows: {groups_more_than_7} (Total rows: {group_counts[group_counts > 7].sum()})')

# Optionally, print the number of rows in each group
print("\nNumber of rows in each group:")
#print(group_counts)

mega_dataframe = merged_df

# Now apply the filtering to mega_dataframe
##################################################################################
### COMPUTE SIDE CHAIN STATISTICS FROM SCORES DF & DO GENTLE/LOGICAL FILTERING ###
##################################################################################

### OLD FILTER FROM GLOBAL STUFF (EVERYTHING SHOULD PASS THIS ONE) ###
plddt_cutoff = 72.5  # in pLDDT units -> usually 70-80
rmsd_cutoff = 1.75  # Å, I think this is Cα (1.5 is typical, 2.0 might be a stretch but okay for big structures)
pae_cutoff = 7.25  # below 5 is harsh but good -> not applicable here
pTM_cutoff = 0.725  # Supposedly above 0.75 is really good & above 0.5 is on the side of probs correct
convergence_cutoff = 0.75  # Get it in a reasonable range

# Adjust column names with suffixes
global_metric_filter_af2_mask = (
    (mega_dataframe['mean_plddt_af2_out'] >= plddt_cutoff) & 
    (mega_dataframe['rmsd_to_input_af2_out'] <= rmsd_cutoff) &
    (mega_dataframe['mean_pae_af2_out'] <= pae_cutoff) & 
    (mega_dataframe['pTMscore_af2_out'] >= pTM_cutoff) &
    (mega_dataframe['af2_convergence_tol_af2_out'] <= convergence_cutoff)
)

filtered_af2_global_scores_df = mega_dataframe[global_metric_filter_af2_mask]
global_af2_filter_rate = float(len(filtered_af2_global_scores_df)) / float(len(mega_dataframe)) * 100.0
print(f'{len(filtered_af2_global_scores_df)} of {len(mega_dataframe)} ({round(global_af2_filter_rate, 3)}%) with plddt >= {plddt_cutoff}, Cα rmsd <= {rmsd_cutoff}Å, convergence <= {convergence_cutoff}Å, pae <= {pae_cutoff}, and pTM >= {pTM_cutoff}')
print('')

### NEW FILTER FROM SIDE CHAIN 2 ###
cat_HIS_bb_rms_of_rmsds_cutoff = 0.85  # Å
cat_HIS_cb_bb_rms_of_rmsds_cutoff = 0.85  # Å
cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff = 0.85  # Å
cat_HIS_imidazole_rms_of_rmsds_cutoff = 1.25  # Å
cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff = 1.25  # Å

# Adjust column names with suffixes
filter_masks = {
    'cat_HIS_bb_rms_of_rmsds_mask': filtered_af2_global_scores_df['cat_HIS_bb_rms_of_rmsds_af2_out'] <= cat_HIS_bb_rms_of_rmsds_cutoff,
    'cat_HIS_cb_bb_rms_of_rmsds_mask': filtered_af2_global_scores_df['cat_HIS_cb_bb_rms_of_rmsds_af2_out'] <= cat_HIS_cb_bb_rms_of_rmsds_cutoff,
    'cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask': filtered_af2_global_scores_df['cat_HIS_ca_cb_ONLY_rms_of_rmsds_af2_out'] <= cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff,
    'cat_HIS_imidazole_rms_of_rmsds_mask': filtered_af2_global_scores_df['cat_HIS_imidazole_rms_of_rmsds_af2_out'] <= cat_HIS_imidazole_rms_of_rmsds_cutoff,
    'cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask': filtered_af2_global_scores_df['cat_HIS_imidazole_tip_Ns_rms_of_rmsds_af2_out'] <= cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff
}

# Calculate pass counts and rates for each filter
filter_results = {filter_name: (mask.sum(), len(filtered_af2_global_scores_df), mask.mean() * 100) for filter_name, mask in filter_masks.items()}

# Overall filtering result using OR logic
cumulative_filter_mask_or = filter_masks['cat_HIS_bb_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_cb_bb_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_imidazole_rms_of_rmsds_mask'] | \
                            filter_masks['cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask']

cumulative_pass_count_or = cumulative_filter_mask_or.sum()
cumulative_total_or = len(filtered_af2_global_scores_df)
cumulative_pass_rate_or = cumulative_pass_count_or / cumulative_total_or * 100

### DO THE PRINT STATEMENT (FANCY) ###
print('########################## SIDE CHAIN RMSD METRICS FILTER ##########################')
for filter_name, (pass_count, total, pass_rate) in filter_results.items():
    specific_criteria = filter_name.replace('_mask', '').replace('_', ' ').replace('cat', 'catalytic').upper()
    cutoff_value = {
        'cat_HIS_bb_rms_of_rmsds_mask': cat_HIS_bb_rms_of_rmsds_cutoff,
        'cat_HIS_cb_bb_rms_of_rmsds_mask': cat_HIS_cb_bb_rms_of_rmsds_cutoff,
        'cat_HIS_ca_cb_ONLY_rms_of_rmsds_mask': cat_HIS_ca_cb_ONLY_rms_of_rmsds_cutoff,
        'cat_HIS_imidazole_rms_of_rmsds_mask': cat_HIS_imidazole_rms_of_rmsds_cutoff,
        'cat_HIS_imidazole_tip_Ns_rms_of_rmsds_mask': cat_HIS_imidazole_tip_Ns_rms_of_rmsds_cutoff
    }[filter_name]
    print(f'{pass_count} of {total} ({pass_rate:.3f}%) have {specific_criteria} <= {cutoff_value}')

# Maintaining the cumulative print statement from the provided script
print('')
print(f'{cumulative_pass_count_or} of {cumulative_total_or} ({cumulative_pass_rate_or:.3f}%) pass at least one filter!')
print('####################################################################################')

### MAKE THE FINAL DATA FRAME ###
final_filtered_df = filtered_af2_global_scores_df[cumulative_filter_mask_or]
filtered_af2_sc_AND_global_scores_df = final_filtered_df

### PRINT OUT COLUMN NAMES USED IN FILTERING ###
filtered_columns = ', '.join([key.replace('_mask', '') for key in filter_masks.keys()])
print(f'Columns used in filtering: {filtered_columns}')

### GROUP BY SECOND SHELL SEQ CLUSTERING ###
grouped = filtered_af2_sc_AND_global_scores_df.groupby('second_shell_seq_clustering')

# Get the number of unique groups
num_unique_groups = grouped.ngroups

# Count the number of rows in each group
group_counts = grouped.size()

# Count how many groups have more than 3, 5, and 7 rows
groups_3 = group_counts[group_counts >= 3].count()
groups_more_than_3 = group_counts[group_counts > 3].count()
print('')
print('METRICS FOR CHECKING WITH BELOW CALCULATIONS/LOGIC:')
print(f'Number of unique groups: {num_unique_groups} (Total rows: {len(filtered_af2_sc_AND_global_scores_df)})')
print(f'Number of groups with 3+ rows: {groups_3} (Total rows: {group_counts[group_counts >= 3].sum()})')
print(f'Number of groups with more than 4+ rows: {groups_more_than_3} (Total rows: {group_counts[group_counts > 3].sum()})')
print('')

### TOP 3 STRUCTURES PER GROUP BY COMBINED SCORE ###
# Normalize rmsd_to_input_af2_out and pTMscore_af2_out within each group
def normalize_series(series):
    return (series - series.min()) / (series.max() - series.min())

filtered_af2_sc_AND_global_scores_df['normalized_rmsd'] = filtered_af2_sc_AND_global_scores_df.groupby('second_shell_seq_clustering')['rmsd_to_input_af2_out'].transform(normalize_series)
filtered_af2_sc_AND_global_scores_df['normalized_pTMscore'] = filtered_af2_sc_AND_global_scores_df.groupby('second_shell_seq_clustering')['pTMscore_af2_out'].transform(normalize_series)

# Create a combined score (50/50 weighting of normalized_rmsd and normalized_pTMscore)
filtered_af2_sc_AND_global_scores_df['combined_score'] = 0.5 * filtered_af2_sc_AND_global_scores_df['normalized_rmsd'] + 0.5 * (1 - filtered_af2_sc_AND_global_scores_df['normalized_pTMscore'])

# Sort by combined_score within each group and keep the top 3 structures, or all structures if there are fewer than 3
def keep_top_3(group):
    return group.nsmallest(3, 'combined_score')

# Ensure all unique groups are represented
shell_filtered_megadata_df = filtered_af2_sc_AND_global_scores_df.groupby('second_shell_seq_clustering').apply(lambda x: x if len(x) < 3 else keep_top_3(x)).reset_index(drop=True)

### PRINT FILTERING RESULTS ###
num_structures_before = len(filtered_af2_sc_AND_global_scores_df)
num_structures_after = len(shell_filtered_megadata_df)
print(f'\nNumber of structures before top 3 filtering: {num_structures_before}')
print(f'Number of structures after top 3 filtering: {num_structures_after}')
print(f'Number of structures filtered out: {num_structures_before - num_structures_after}')
print('')

### GROUP BY SECOND SHELL SEQ CLUSTERING ###
grouped = shell_filtered_megadata_df.groupby('second_shell_seq_clustering')

# Get the number of unique groups
num_unique_groups = grouped.ngroups

# Count the number of rows in each group
group_counts = grouped.size()

# Count how many groups have 3 rows
groups_3 = group_counts[group_counts == 3].count()
groups_more_than_3 = group_counts[group_counts > 3].count()
print(f'Number of unique groups: {num_unique_groups}')
print(f'Number of groups with 3 rows: {groups_3} (Total rows: {group_counts[group_counts == 3].sum()})')
print(f'Number of groups with more than 3 rows: {groups_more_than_3} (Total rows: {group_counts[group_counts > 3].sum()})')

### CREATE FASTMPNN_CLUSTERING COLUMN ###
def extract_fastmpnn_clustering(seq_clustering):
    return seq_clustering.split('_model_')[0]

shell_filtered_megadata_df['fastmpnn_clustering'] = shell_filtered_megadata_df['second_shell_seq_clustering'].apply(extract_fastmpnn_clustering)

# Find the number of unique fastmpnn groups
num_unique_fastmpnn_groups = shell_filtered_megadata_df['fastmpnn_clustering'].nunique()
print('')
print(f'Number of unique fastmpnn groups: {num_unique_fastmpnn_groups}')

# Optionally, show the final filtered DataFrame
print(shell_filtered_megadata_df.head())

AF2 DataFrame before merge:
(21, 30)

APO Relax DataFrame before merge:
(13, 19)

Merged DataFrame:
(13, 48)

Total number of rows in the merged DataFrame: 13
Number of unique groups: 8
Number of groups with more than 3 rows: 0 (Total rows: 0)
Number of groups with more than 5 rows: 0 (Total rows: 0)
Number of groups with more than 7 rows: 0 (Total rows: 0)

Number of rows in each group:
12 of 13 (92.308%) with plddt >= 72.5, Cα rmsd <= 1.75Å, convergence <= 0.75Å, pae <= 7.25, and pTM >= 0.725

########################## SIDE CHAIN RMSD METRICS FILTER ##########################
11 of 12 (91.667%) have CATALYTIC HIS BB RMS OF RMSDS <= 0.85
10 of 12 (83.333%) have CATALYTIC HIS CB BB RMS OF RMSDS <= 0.85
10 of 12 (83.333%) have CATALYTIC HIS CA CB ONLY RMS OF RMSDS <= 0.85
0 of 12 (0.000%) have CATALYTIC HIS IMIDAZOLE RMS OF RMSDS <= 1.25
0 of 12 (0.000%) have CATALYTIC HIS IMIDAZOLE TIP NS RMS OF RMSDS <= 1.25

12 of 12 (100.000%) pass at least one filter!
#############################

/tmp/ipykernel_1763/346897955.py:219: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  shell_filtered_megadata_df = filtered_af2_sc_AND_global_scores_df.groupby('second_shell_seq_clustering').apply(lambda x: x if len(x) < 3 else keep_top_3(x)).reset_index(drop=True)


In [35]:
###################################################################################################
### GET RID OF THE SHITTY SEQUENCES TO START FILTERING DOWN (IF ALL ARE SHIT TAKE THE BEST ONE) ###
###################################################################################################

### INPUTS ###
df_sorted = shell_filtered_megadata_df
pdb_column = 'pdb_path_af2_out'
grouping_column = 'fastmpnn_clustering'
ptm_column = 'pTMscore_af2_out'
rmsd_column = 'rmsd_to_input_af2_out'
similarity_threshold = 0.975 # SEQUENCE SIMILARITY TO DEFINE A CLUSTER

# Function to extract amino acid sequences from PDB files, considering only the first sequence if multiple are present
def extract_sequences(pdb_paths):
    sequences = {}
    pdb_parser = PDBParser(QUIET=True)
    pp_builder = PPBuilder()
    
    for pdb_path in pdb_paths:
        try:
            structure = pdb_parser.get_structure(pdb_path, pdb_path)
            for pp in pp_builder.build_peptides(structure):
                seq = pp.get_sequence()
                sequences[pdb_path] = str(seq)
                break  # Only take the first sequence
        except Exception as e:
            print(f"Failed to read {pdb_path}: {e}")
    
    if sequences:
        random_path = random.choice(list(sequences.keys()))
        print(f"Randomly selected sequence from {random_path}: {sequences[random_path]}")
    else:
        print("No sequences extracted.")
    
    return sequences

# Function to calculate similarity between two sequences
def calculate_similarity(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1, seq2)
    best_alignment = alignments[0]
    matches = best_alignment[2]
    similarity = matches / max(len(seq1), len(seq2))
    return similarity

# Function to cluster sequences by similarity
def cluster_sequences(group, threshold):
    clusters = []
    for index, row in group.iterrows():
        seq = row['sequence']
        placed = False
        for cluster in clusters:
            if calculate_similarity(seq, cluster['representative']['sequence']) >= threshold:
                cluster['members'].append(row)
                placed = True
                break
        if not placed:
            clusters.append({'representative': row, 'members': [row]})
    return clusters

# Function to flag sequences with repetitive amino acids, excessive same-charge residues, long hydrophobic stretches, or specific patterns
def flag_sequences(seq):
    repetitive_pattern = re.compile(r"(.)\1{4,}")  # flag sequences with four or more consecutive identical amino acids
    negative_charged_pattern = re.compile(r"(D{5,}|E{5,}|(DE){5,})")  # flag sequences with four or more consecutive negatively charged residues
    positive_charged_pattern = re.compile(r"(K{5,}|R{5,}|(KR){5,})")  # flag sequences with four or more consecutive positively charged residues
    hydrophobic_pattern = re.compile(r"[AVILMFYWP]{7,}")  # flag sequences with six or more consecutive hydrophobic residues
    specific_pattern = re.compile(r"(AANDENYALAA)")  # flag sequences with RR or KR
    
    repetitive_match = repetitive_pattern.search(seq)
    negative_charged_match = negative_charged_pattern.search(seq)
    positive_charged_match = positive_charged_pattern.search(seq)
    hydrophobic_match = hydrophobic_pattern.search(seq)
    specific_match = specific_pattern.search(seq)
    
    is_repetitive = bool(repetitive_match)
    is_negatively_charged = bool(negative_charged_match)
    is_positively_charged = bool(positive_charged_match)
    is_hydrophobic = bool(hydrophobic_match)
    is_specific = bool(specific_match)
    
    flagged_parts = []
    violations = 0
    if is_repetitive:
        flagged_parts.append(f"Repetitive: {repetitive_match.group()}")
        violations += 1
    if is_negatively_charged:
        flagged_parts.append(f"Negatively Charged: {negative_charged_match.group()}")
        violations += 1
    if is_positively_charged:
        flagged_parts.append(f"Positively Charged: {positive_charged_match.group()}")
        violations += 1
    if is_hydrophobic:
        flagged_parts.append(f"Hydrophobic: {hydrophobic_match.group()}")
        violations += 1
    if is_specific:
        flagged_parts.append(f"Specific Pattern: {specific_match.group()}")
        violations += 1

    if flagged_parts:
        flagged_str = ", ".join(flagged_parts)
        print(f"Flagging sequence: {seq} - {flagged_str}")
    
    return is_repetitive or is_negatively_charged or is_positively_charged or is_hydrophobic or is_specific, flagged_parts, violations

# Main function to process the DataFrame
def process_dataframe(df_sorted, similarity_threshold=0.85):
    print("Starting process...")
    print(f"Total rows before filtering: {len(df_sorted)}")
    print(f"Number of unique {grouping_column} before filtering: {df_sorted[grouping_column].nunique()}")
    
    group_sizes = df_sorted.groupby(grouping_column).size()
    df_filtered_groups = df_sorted[df_sorted[grouping_column].isin(group_sizes[group_sizes >= 1].index)].copy()
    print(f"Filtered groups with 1 or more entries: {len(df_filtered_groups)}")
    print(f"Number of unique {grouping_column} after filtering groups with 1 or more entries: {df_filtered_groups[grouping_column].nunique()}")

    pdb_paths = df_filtered_groups[pdb_column].unique()
    sequences = extract_sequences(pdb_paths)
    df_filtered_groups['sequence'] = df_filtered_groups[pdb_column].map(sequences)
    df_filtered_groups['sequence_to_repair'] = np.nan

    final_df = pd.DataFrame()
    subgroup_counts = {}
    flagged_repetitive_sequences = []
    flagged_charged_sequences = []
    flagged_hydrophobic_sequences = []
    flagged_specific_sequences = []

    if df_filtered_groups.empty:
        print("No groups with 1 or more entries. Exiting...")
        return pd.DataFrame()

    for group_value, group in df_filtered_groups.groupby(grouping_column):
        print('')
        print(f"Processing {grouping_column} {group_value} with {len(group)} entries")
        group['flag'], group['flagged_parts'], group['violations'] = zip(*group['sequence'].apply(lambda seq: flag_sequences(seq) if isinstance(seq, str) else (True, [], 0)))
        
        # Check if all entries are flagged
        if group['flag'].all():
            print(f"All sequences in group {group_value} are flagged.")
            group['pTM_rmsd_ratio'] = group[ptm_column] / group[rmsd_column]
            min_violations = group['violations'].min()
            group = group[group['violations'] == min_violations]  # Keep rows with the least violations
            if not group.empty:
                selected = group.loc[group['pTM_rmsd_ratio'].idxmax()]
                flagged_part_str = ", ".join(selected['flagged_parts'])
                selected['sequence_to_repair'] = flagged_part_str
                final_df = pd.concat([final_df, pd.DataFrame([selected])], ignore_index=True)
            continue

        flagged_repetitive_sequences += group[group['flag']]['sequence'].tolist()
        flagged_charged_sequences += group[group['flag']]['sequence'].tolist()
        flagged_hydrophobic_sequences += group[group['flag']]['sequence'].tolist()
        flagged_specific_sequences += group[group['flag']]['sequence'].tolist()
        group = group[~group['flag']]

        if group.empty:
            print(f"No valid sequences left after flagging in group {group_value}")
            print('')
            continue

        clusters = cluster_sequences(group, similarity_threshold)
        if not clusters:
            print(f"No clusters formed in group {group_value}")
            print('')
            continue

        subgroup_counts[group_value] = len(clusters)
        for cluster in clusters:
            # SELECT BEST CLUSTER BASED ON LOWEST RMSD
            selected = min(cluster['members'], key=lambda x: x[rmsd_column])
            final_df = pd.concat([final_df, pd.DataFrame([selected])], ignore_index=True)

    print("Final processing complete.")
    print(f"Total sequences flagged for consecutive charged residues: {len(set(flagged_charged_sequences))}")
    print(f"Total sequences flagged for repetitive amino acids: {len(set(flagged_repetitive_sequences))}")
    print(f"Total sequences flagged for hydrophobic stretches: {len(set(flagged_hydrophobic_sequences))}")
    print(f"Total sequences flagged for specific patterns: {len(set(flagged_specific_sequences))}")
    print(f"Number of rows in the final DataFrame: {len(final_df)}")
    print(f"Number of unique {grouping_column} after filtering: {final_df[grouping_column].nunique() if not final_df.empty else 0}")
    print(f"Number of sequences with exact duplicates in the final DataFrame: {final_df.duplicated(subset=['sequence']).sum()}")
    print('')
    
    # Print the number of subgroups for each group
    if subgroup_counts:
        for group_value, count in subgroup_counts.items():
            print(f"{grouping_column} {group_value} has {count} files")
    else:
        print("No data available in the final DataFrame after processing.")
    
    return final_df

# Print column names to verify the presence of grouping_column
print("Columns in df_sorted:", df_sorted.columns)

# Check for the presence of grouping_column in the DataFrame
if grouping_column not in df_sorted.columns:
    raise ValueError(f"DataFrame does not contain '{grouping_column}'. Please check your DataFrame columns.")

# Proceed with the rest of the script if the column check is fine
final_df = process_dataframe(df_sorted, similarity_threshold)

# Optionally, display the resulting DataFrame
if not final_df.empty:
    print(f"Number of rows in the new DataFrame: {len(final_df)}")
    if grouping_column in final_df.columns:
        print(f"Number of unique {grouping_column} after filtering: {final_df[grouping_column].nunique()}")
    else:
        print("No grouping column in final DataFrame; possibly all data was filtered out.")

Columns in df_sorted: Index(['af2_json_path_af2_out', 'pdb_path_af2_out', 'mean_plddt_af2_out',
       'rmsd_to_input_af2_out', 'mean_pae_intra_chain_af2_out',
       'mean_pae_af2_out', 'pTMscore_af2_out', 'af2_convergence_tol_af2_out',
       'af2_elapsed_folding_time_af2_out', 'cat_HIS1_bb_rmsd_af2_out',
       'cat_HIS1_cb_bb_rmsd_af2_out', 'cat_HIS1_imidazole_rmsd_af2_out',
       'cat_HIS1_ca_cb_rmsd_af2_out', 'cat_HIS1_imid_sc_nitrogen_rmsd_af2_out',
       'cat_HIS2_bb_rmsd_af2_out', 'cat_HIS2_cb_bb_rmsd_af2_out',
       'cat_HIS2_imidazole_rmsd_af2_out', 'cat_HIS2_ca_cb_rmsd_af2_out',
       'cat_HIS2_imid_sc_nitrogen_rmsd_af2_out', 'cat_HIS3_bb_rmsd_af2_out',
       'cat_HIS3_cb_bb_rmsd_af2_out', 'cat_HIS3_imidazole_rmsd_af2_out',
       'cat_HIS3_ca_cb_rmsd_af2_out', 'cat_HIS3_imid_sc_nitrogen_rmsd_af2_out',
       'cat_HIS_bb_rms_of_rmsds_af2_out', 'cat_HIS_cb_bb_rms_of_rmsds_af2_out',
       'cat_HIS_ca_cb_ONLY_rms_of_rmsds_af2_out',
       'cat_HIS_imidazole_rms_of_rmsds_

In [36]:
final_df.to_csv(f'{important_dfs_dir}RFD2_sequence_clustered_post_fastmpnn_af2_pre_placer.csv', index=False)
# num1:f'{important_dfs_dir}RFD2_sequence_clustered_post_fastmpnn_af2_pre_placer.csv'
# num2: f'{important_dfs_dir}inpainting_sequence_clustered_post_fastmpnn_af2_pre_placer.csv'

## Copy AF2 + LIGAND into PLACER Input Directory

In [37]:
######################################################################################################################
### COPY APO RELAXED AF2 STRUCTURES WITH LIGAND COORDS INTO FILE (WE NEED TO PULL ALIGNED APO STRUCTURES FOR THIS) ###
######################################################################################################################

align_af2_out_directory = os.path.join(f"{af2_out_dir}", "predesigned_RFD2_i2", "filtered_structures/alignment/")
# num1: predesigned_RFD2_i2
# num2: predesigned_inpainting_i2

# Define the destination directory
destination_dir = os.path.join(placer_input_structures_dir, "RFD2", "holo")
# num1: RFD2
# num2: inpainting
os.makedirs(destination_dir, exist_ok=True)

# Get the list of files to copy from the DataFrame
files_to_copy = shell_filtered_megadata_df['pdb_path_af2_apo_relax_out'].tolist()
total_files = len(files_to_copy)

print(f"Total number of files to copy: {total_files}")

# Initialize counters
copied_files_count = 0
missing_files = []

# Copy the files and print updates after every 1000 files
for i, file_path in enumerate(files_to_copy):
    # Extract the basename and remove the .pdb extension
    base_name = os.path.basename(file_path).replace('.pdb', '')
    
    # Search for the corresponding files in align_af2_out_directory
    search_pattern = os.path.join(align_af2_out_directory, f"{base_name}*.pdb")
    matched_files = glob.glob(search_pattern)
    
    if matched_files:
        for matched_file in matched_files:
            shutil.copy(matched_file, destination_dir)
            copied_files_count += 1
    else:
        missing_files.append(search_pattern)
    
    if (i + 1) % 1000 == 0:
        remaining_files = total_files - (i + 1)
        print(f"Copied {copied_files_count} files. {remaining_files} files remaining.")

# Final update after all files are copied
print(f"All {copied_files_count} files have been copied to {destination_dir}.")
print(f"Total number of files that were not found: {len(missing_files)}")
if missing_files:
    print("Files not found:")
    for missing in missing_files:
        print(missing)

Total number of files to copy: 12
All 12 files have been copied to ./output/placer_input_structures/RFD2/holo.
Total number of files that were not found: 0


# **VI. PLACER**

## Run PLACER

In [38]:
###############################################
### GENERATING SUBSTRATE-BINDING STRUCTURES ###
###############################################

input_pdb_dir = os.path.join(placer_input_structures_dir, "RFD2", "holo")
#num1: f"{placer_input_structures_dir}RFD2"
#num2: f"{placer_input_structures_dir}inpainting"

subst_pdb_dir = f"{placer_input_structures_dir}RFD2/subst_only/"
#num1: f"{placer_input_structures_dir}RFD2/subst_only/"
#num1: f"{placer_input_structures_dir}inpainting/subst_only/"
os.makedirs(subst_pdb_dir, exist_ok=True)

subst_atom_list = ["remove", "ZN1", "O1", "H1"]

script = '{scripts_dir}modify_atoms_in_pdb.py'

input_list = glob.glob(os.path.join(input_pdb_dir, "*.pdb"))
print(f"Total number of files to generate substrate-binding structures: {len(input_list)}")

input_pdb_list = glob.glob(os.path.join(input_pdb_dir, "*.pdb"))
start_time = time.time()
for i, input_pdb in enumerate(input_pdb_list):
    if np.log10(i+1)%1 == 0:        
        elapsed_time = time.time() - start_time
        time_per_row = elapsed_time / (i+1)
        estimated_time_remaining = time_per_row * (len(input_pdb_list) - i-1)
        print(f"{i+1} th PDB processed, {len(input_pdb_list) - i+1} remaining. "
              f"Elapsed time: {elapsed_time:.2f} s. Estimated time left: {estimated_time_remaining:.2f} s.")
    elif i+1 == len(input_pdb_list):
        print (f"{i+1} th PDB processed. Done.")
        
    bn = os.path.basename(input_pdb)
    subst_pdb_out = os.path.join(subst_pdb_dir, bn.replace(".pdb", "_subst.pdb"))
    
    for atom_list in [subst_atom_list]:
        os.system(f"python {script} --input {input_pdb} --output {subst_pdb_out} --atoms_to_remove {' '.join(atom_list[1:])}")

Total number of files to generate substrate-binding structures: 12
1 th PDB processed, 13 remaining. Elapsed time: 0.00 s. Estimated time left: 0.00 s.
10 th PDB processed, 4 remaining. Elapsed time: 0.49 s. Estimated time left: 0.10 s.
12 th PDB processed. Done.


In [41]:
########################################
### SUBSTRATE-BINDING DESIGN PLACER ###
########################################

# IMPORTANT INPUTS 
placer_input_structures = f"{placer_input_structures_dir}RFD2/subst_only/"
#num1: f"{placer_input_structures_dir}RFD2/subst_only/"
#num1: f"{placer_input_structures_dir}inpainting/subst_only/"

placer_out = f'{placer_out_dir}RFD2/subst_only/' # Update occassionally
#num1: f'{placer_out_dir}RFD2/subst_only/'
#num1: f'{placer_out_dir}inpainting/subst_only/'

num_samples = 40

# Mostly Stay constant
os.makedirs(placer_out,exist_ok=True)
script = '{software_dir}PLACER/run_PLACER.py'

# Make PLACER Commands
print ("Run below commands:")
for pdb in glob.glob(f'{placer_input_structures}*pdb'): 
    bn = os.path.basename(pdb).replace('.pdb','')
    cmd_txt = (f'python {script} ' #KEEP python
               f'-f {pdb} '
               f'-n {num_samples} '
               f'--odir {placer_out} '
               f'--ocsv {placer_out}{bn}.csv')
    print (cmd_txt)

Run below commands:
python {software_dir}PLACER/run_PLACER.py -f ./output/placer_input_structures/RFD2/subst_only/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_1_model_4_ptm_seed_0_unrelaxed_relaxedAPO_PSZ_subst.pdb -n 40 --odir ./output/placer_out/RFD2/subst_only/ --ocsv ./output/placer_out/RFD2/subst_only/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_1_model_4_ptm_seed_0_unrelaxed_relaxedAPO_PSZ_subst.csv
python {software_dir}PLACER/run_PLACER.py -f ./output/placer_input_structures/RFD2/subst_only/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_8_T0.1_0_model_4_ptm_seed_0_unrelaxed_relaxedAPO_PSZ_subst.pdb -n 40 --odir ./output/placer_out/RFD2/subst_only/ --ocsv ./output/placer_out/RFD2/subst_only/zn_hydrolase_PSZ_H_H_H_1_63_0_1-atomized-bb-False_0_10_model_4_ptm_seed_0_unrelaxed_PSZ_0_8_T0.1_0_model_4_ptm_seed_0_unrelaxed_relaxedAPO_PSZ_subst.csv
python {

## PLACER Analysis Run

In [ ]:
#####################################################
### PLACER ANALYSIS FOR SUBSTRATE-BINDING DESIGN ###
#####################################################

script = '{scripts_dir}process_placer.py'

multiple_ligand_names = {"PSZ"}
lig_atom = "C1 C2 C3 C4 C5 C6 C7 C8 C9 C10 C11 C12 C13 C14 C15 C16 C17 C18 O2 O3 O4 O5"

params_path_DIR = f"{invrot_dir}COMBINED_PHENYLACETATE_HHH/COMBINED_PARAMS/"
params_path = glob.glob(os.path.join(params_path_DIR, "*.params"))

placer_input_path = f"{placer_input_structures_dir}inpainting/subst_only/"
#num1: f"{placer_input_structures_dir}RFD2/subst_only/"
#num1: f"{placer_input_structures_dir}inpainting/subst_only/"

placer_output_path = f'{placer_out_dir}inpainting/subst_only/' # Update occassionally
#num1: f'{placer_out_dir}RFD2/subst_only/'
#num1: f'{placer_out_dir}inpainting/subst_only/'

for placer_output_pdb in glob.glob(os.path.join(placer_output_path, "*.pdb")):
    bn = os.path.basename(placer_output_pdb)
    placer_input = os.path.join(placer_input_path, bn.replace("_model.pdb", ".pdb"))
    placer_output_csv = os.path.join(placer_output_path, bn.replace("_model.pdb", ".csv"))
    score_output = os.path.join(placer_output_path, bn.replace("_model.pdb", "_updated_score.csv"))

    lig_name = list(filter(lambda lig: lig in bn, multiple_ligand_names))[0]

    cmd_txt = (f'python {script} '
           f'--pdb {placer_input} '
           f'--scorefile {placer_output_csv} '
           f'--lig_name {lig_name} '
           f'--lig_atom {lig_atom} '    
           f'--params {" ".join(params_path)} '
           f'--placer_pdb_path {placer_output_path} '
           f'--scorefile_out {score_output} \n')
    print(cmd_txt)  

## PLACER Analysis Substrate-binding Designs Stats

In [ ]:
##################################################
### COMBINE THE ABOVE TWO DATA FRAMES AND SAVE ###
##################################################

# Define the paths
placer_subst_input_path = f"{placer_input_structures_dir}inpainting/subst_only/"
#num1: f"{placer_input_structures_dir}RFD2/subst_only/"
#num2: f"{placer_input_structures_dir}inpainting/subst_only/"

placer_subst_output_path = f'{placer_out_dir}inpainting/subst_only/' # Update occassionally
#num1: f'{placer_out_dir}RFD2/subst_only/'
#num2: f'{placer_out_dir}inpainting/subst_only/'

file_paths = glob.glob(os.path.join(placer_subst_output_path, "*updated_score.csv"))
print("Number of found updated score files:", len(file_paths))

df_list = []
file_names = []
for fi in file_paths:
    df = pd.read_csv(fi, sep=",")
    if not df.empty:
        df_list.append(df)
        file_names.append(os.path.basename(fi))
placer_subst_df = pd.concat(df_list, ignore_index=True)
print("Size of placer_subst_df", len(placer_subst_df))

# Remove unnamed columns if they exist
placer_subst_df = placer_subst_df.loc[:, ~placer_subst_df.columns.str.contains('^Unnamed')]

# Add the 'name' column
placer_subst_df["name"] = placer_subst_df["description"].str[:-6]

# Create new columns
subst_top_pdb_paths = [os.path.join(placer_subst_output_path, str(el).replace("_subst", "_subst_model.pdb")) for el in placer_subst_df['description'].tolist()]
pdb_path_subst_placer = subst_top_pdb_paths
pdb_path_subst_placer_INPUT = [os.path.join(placer_subst_input_path, os.path.basename(path).replace("_model.pdb", ".pdb")) for path in subst_top_pdb_paths]

# Add new columns to merged_df
placer_subst_df["pdb_path_subst_placer"] = pdb_path_subst_placer
placer_subst_df["pdb_path_subst_placer_INPUT"] = pdb_path_subst_placer_INPUT

# Define the save path
save_path = f'{important_dfs_dir}inpainting_placer_subst_combined_df.csv'
#num1: RFD2
#num2: inpainting

# Save the new dataframe
placer_subst_df.to_csv(save_path, index=False)

# Print the save location
print(f"Dataframe saved to: {save_path}")